In [1]:
!pwd
import sys

C:\Users\81908\jupyter_notebook\poetry_work\tfgpu\01_MoA_compe\notebook\base_line\20201105


In [2]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline
sys.executable

'c:\\users\\81908\\appdata\\local\\pypoetry\\cache\\virtualenvs\\tfgpu-ehdmne1y-py3.8\\scripts\\python.exe'

In [3]:
import datetime
import os
import pathlib
import sys
import warnings
import numpy as np
import pandas as pd
from sklearn.preprocessing import RobustScaler, StandardScaler

warnings.filterwarnings("ignore")
pd.set_option("display.max_columns", None)

sys.path.append(r"C:\Users\81908\jupyter_notebook\poetry_work\tfgpu\01_MoA_compe\code")
import mlp_tf, datasets, util

Tensorflow version 2.3.1


In [4]:
def load_data():
    (
        train,
        train_targets,
        test,
        sample_submission,
        train_targets_nonscored,
        train_drug,
    ) = datasets.load_orig_data()
    train, train_targets, test, train_targets_nonscored = datasets.mapping_and_filter(
        train, train_targets, test, train_targets_nonscored
    )
    return (
        train,
        train_targets,
        test,
        sample_submission,
        train.columns[2:],
        train_targets_nonscored,
    )

## 条件かえてoof_log_loss確認

In [5]:
(
    train,
    train_targets,
    test,
    sample_submission,
    features,
    train_targets_nonscored,
) = load_data()

In [6]:
train_targets

,sig_id,5-alpha_reductase_inhibitor,11-beta-hsd1_inhibitor,acat_inhibitor,acetylcholine_receptor_agonist,acetylcholine_receptor_antagonist,acetylcholinesterase_inhibitor,adenosine_receptor_agonist,adenosine_receptor_antagonist,adenylyl_cyclase_activator,adrenergic_receptor_agonist,adrenergic_receptor_antagonist,akt_inhibitor,aldehyde_dehydrogenase_inhibitor,alk_inhibitor,ampk_activator,analgesic,androgen_receptor_agonist,androgen_receptor_antagonist,anesthetic_-_local,angiogenesis_inhibitor,angiotensin_receptor_antagonist,anti-inflammatory,antiarrhythmic,antibiotic,anticonvulsant,antifungal,antihistamine,antimalarial,antioxidant,antiprotozoal,antiviral,apoptosis_stimulant,aromatase_inhibitor,atm_kinase_inhibitor,atp-sensitive_potassium_channel_antagonist,atp_synthase_inhibitor,atpase_inhibitor,atr_kinase_inhibitor,aurora_kinase_inhibitor,autotaxin_inhibitor,bacterial_30s_ribosomal_subunit_inhibitor,bacterial_50s_ribosomal_subunit_inhibitor,bacterial_antifolate,bacterial_cell_wall_synthesis_inhibitor,bacterial_dna_gyrase_inhibitor,bacterial_dna_inhibitor,bacterial_membrane_integrity_inhibitor,bcl_inhibitor,bcr-abl_inhibitor,benzodiazepine_receptor_agonist,beta_amyloid_inhibitor,bromodomain_inhibitor,btk_inhibitor,calcineurin_inhibitor,calcium_channel_blocker,cannabinoid_receptor_agonist,cannabinoid_receptor_antagonist,carbonic_anhydrase_inhibitor,casein_kinase_inhibitor,caspase_activator,catechol_o_methyltransferase_inhibitor,cc_chemokine_receptor_antagonist,cck_receptor_antagonist,cdk_inhibitor,chelating_agent,chk_inhibitor,chloride_channel_blocker,cholesterol_inhibitor,cholinergic_receptor_antagonist,coagulation_factor_inhibitor,corticosteroid_agonist,cyclooxygenase_inhibitor,cytochrome_p450_inhibitor,dihydrofolate_reductase_inhibitor,dipeptidyl_peptidase_inhibitor,diuretic,dna_alkylating_agent,dna_inhibitor,dopamine_receptor_agonist,dopamine_receptor_antagonist,egfr_inhibitor,elastase_inhibitor,erbb2_inhibitor,estrogen_receptor_agonist,estrogen_receptor_antagonist,faah_inhibitor,farnesyltransferase_inhibitor,fatty_acid_receptor_agonist,fgfr_inhibitor,flt3_inhibitor,focal_adhesion_kinase_inhibitor,free_radical_scavenger,fungal_squalene_epoxidase_inhibitor,gaba_receptor_agonist,gaba_receptor_antagonist,gamma_secretase_inhibitor,glucocorticoid_receptor_agonist,glutamate_inhibitor,glutamate_receptor_agonist,glutamate_receptor_antagonist,gonadotropin_receptor_agonist,gsk_inhibitor,hcv_inhibitor,hdac_inhibitor,histamine_receptor_agonist,histamine_receptor_antagonist,histone_lysine_demethylase_inhibitor,histone_lysine_methyltransferase_inhibitor,hiv_inhibitor,hmgcr_inhibitor,hsp_inhibitor,igf-1_inhibitor,ikk_inhibitor,imidazoline_receptor_agonist,immunosuppressant,insulin_secretagogue,insulin_sensitizer,integrin_inhibitor,jak_inhibitor,kit_inhibitor,laxative,leukotriene_inhibitor,leukotriene_receptor_antagonist,lipase_inhibitor,lipoxygenase_inhibitor,lxr_agonist,mdm_inhibitor,mek_inhibitor,membrane_integrity_inhibitor,mineralocorticoid_receptor_antagonist,monoacylglycerol_lipase_inhibitor,monoamine_oxidase_inhibitor,monopolar_spindle_1_kinase_inhibitor,mtor_inhibitor,mucolytic_agent,neuropeptide_receptor_antagonist,nfkb_inhibitor,nicotinic_receptor_agonist,nitric_oxide_donor,nitric_oxide_production_inhibitor,nitric_oxide_synthase_inhibitor,norepinephrine_reuptake_inhibitor,nrf2_activator,opioid_receptor_agonist,opioid_receptor_antagonist,orexin_receptor_antagonist,p38_mapk_inhibitor,p-glycoprotein_inhibitor,parp_inhibitor,pdgfr_inhibitor,pdk_inhibitor,phosphodiesterase_inhibitor,phospholipase_inhibitor,pi3k_inhibitor,pkc_inhibitor,potassium_channel_activator,potassium_channel_antagonist,ppar_receptor_agonist,ppar_receptor_antagonist,progesterone_receptor_agonist,progesterone_receptor_antagonist,prostaglandin_inhibitor,prostanoid_receptor_antagonist,proteasome_inhibitor,protein_kinase_inhibitor,protein_phosphatase_inhibitor,protein_synthesis_inhibitor,protein_tyrosine_kinase_inhibitor,radiopaque_medium,raf_inhibitor,ras_gtpase_inhib

In [7]:
train_targets_nonscored

,abc_transporter_expression_enhancer,abl_inhibitor,ace_inhibitor,acetylcholine_release_enhancer,adenosine_deaminase_inhibitor,adenosine_kinase_inhibitor,adenylyl_cyclase_inhibitor,age_inhibitor,alcohol_dehydrogenase_inhibitor,aldehyde_dehydrogenase_activator,aldose_reductase_inhibitor,alpha_mannosidase_inhibitor,ampk_inhibitor,androgen_biosynthesis_inhibitor,angiotensin_receptor_agonist,antacid,anthelmintic,antihypertensive,antimetabolite,antipruritic,antirheumatic_drug,antiseptic,antispasmodic,antithyroid_agent,antitussive,anxiolytic,ap_inhibitor,apolipoprotein_expression_enhancer,apoptosis_inhibitor,arf_inhibitor,aryl_hydrocarbon_receptor_agonist,aryl_hydrocarbon_receptor_antagonist,aspartic_protease_inhibitor,atherogenesis_inhibitor,atherosclerosis_formation_inhibitor,atp-sensitive_potassium_channel_agonist,atp-sensitive_potassium_channel_inhibitor,atp_channel_activator,atp_channel_blocker,atp_citrase_lyase_inhibitor,autophagy_inducer,axl_kinase_inhibitor,bacterial_atpase_inhibitor,bacterial_permeability_inducer,bacterial_protein_synthesis_inhibitor,benzodiazepine_receptor_antagonist,beta_catenin_inhibitor,beta_lactamase_inhibitor,beta_secretase_inhibitor,big1_inhibitor,bile_acid,biliverdin_reductase_a_activator,bone_resorption_inhibitor,botulin_neurotoxin_inhibitor,bradykinin_receptor_antagonist,breast_cancer_resistance_protein_inhibitor,bronchodilator,calcitonin_antagonist,calcium_channel_activator,calmodulin_inhibitor,calpain_inhibitor,camp_stimulant,capillary_stabilizing_agent,car_agonist,car_antagonist,carboxylesterase_inhibitor,carcinogen,cardiac_glycoside,carnitine_palmitoyltransferase_inhibitor,caspase_inhibitor,cathepsin_inhibitor,cc_chemokine_receptor_agonist,cdc_inhibitor,cdk_expression_enhancer,cell_cycle_inhibitor,cell_proliferation_inhibitor,ceramidase_inhibitor,cftr_channel_agonist,cftr_channel_antagonist,chitin_inhibitor,chloride_channel_activator,choleretic_agent,cholinergic_receptor_agonist,cholinesterase_inhibitor,clk_inhibitor,coenzyme_a_precursor,collagenase_inhibitor,collapsin_response_mediator_protein_stimulant,coloring_agent,complement_antagonist,complement_inhibitor,contraceptive_agent,contrast_agent,corticosteroid_antagonist,cyclin_d_inhibitor,cysteine_peptidase_inhibitor,cytidine_deaminase_inhibitor,cytokine_production_inhibitor,dehydrogenase_inhibitor,deubiquitinase_inhibitor,diacylglycerol_kinase_inhibitor,diacylglycerol_o_acyltransferase_inhibitor,differentiation_inducer,dihydroorotate_dehydrogenase_inhibitor,dihydropteroate_synthase_inhibitor,dihydropyrimidine_dehydrogenase_inhibitor,dna_dependent_protein_kinase_inhibitor,dna_methyltransferase_inhibitor,dna_polymerase_inhibitor,dna_repair_enzyme_inhibitor,dna_synthesis_inhibitor,dopamine_release_enhancer,dot1l_inhibitor,dynamin_inhibitor,dyrk_inhibitor,dystrophin_stimulant,endothelin_receptor_antagonist,enkephalinase_inhibitor,ephrin_inhibitor,epoxide_hydolase_inhibitor,etv1_inhibitor,eukaryotic_translation_initiation_factor_inhibitor,exportin_antagonist,fabi_inhibitor,farnesyl_pyrophosphate_synthase_inhibitor,fatty_acid_receptor_antagonist,fatty_acid_synthase_inhibitor,folate_receptor_ligand,free_fatty_acid_receptor_agonist,"fungal_1,3-beta-d-glucan_synthase_inhibitor",fungal_ergosterol_inhibitor,fungal_lanosterol_demethylase_inhibitor,fxr_agonist,fxr_antagonist,g_protein-coupled_receptor_agonist,g_protein-coupled_receptor_antagonist,g_protein_signaling_inhibitor,gaba_gated_chloride_channel_blocker,gaba_receptor_modulator,gaba_uptake_inhibitor,gap_junction_modulator,gastrin_inhibitor,gat_inhibitor,glcnac_phosphotransferase_inhibitor,gli_antagonist,glp_receptor_agonist,glucagon_receptor_antagonist,glucocorticoid_receptor_antagonist,glucokinase_activator,glucokinase_inhibitor,gluconeogenesis_inhibitor,glucose_dependent_insulinotropic_receptor_agonist,glucosidase_inhibitor,glutamate_receptor_modulator,glutathione_peroxidase_agonist,glutathione_reductase_(nadph)_activators,glutathione_transferase_inhibitor,glycine_receptor_antagonist,glycine_tran

In [8]:
features

Index(['cp_time', 'cp_dose', 'g-0', 'g-1', 'g-2', 'g-3', 'g-4', 'g-5', 'g-6',
       'g-7',
       ...
       'c-90', 'c-91', 'c-92', 'c-93', 'c-94', 'c-95', 'c-96', 'c-97', 'c-98',
       'c-99'],
      dtype='object', length=874)

In [9]:
logger = util.Logger("./")
# デバッグ用にデータ減らすか
DEBUG = False
#DEBUG = True
if DEBUG:
    mlp_tf.FOLDS = 2  # cvの数
    mlp_tf.EPOCHS = 2
    logger.info(f"### DEBUG: FOLDS:{mlp_tf.FOLDS}, EPOCHS:{mlp_tf.EPOCHS} ###")
else:
    logger.info(f"### HONBAN: FOLDS:{mlp_tf.FOLDS}, EPOCHS:{mlp_tf.EPOCHS} ###")

[2020-11-06 10:11:57] - ### HONBAN: FOLDS:5, EPOCHS:80 ###


## 動作確認

In [10]:
# 1モデルだけで学習推論できるかテスト
(
    train,
    train_targets,
    test,
    sample_submission,
    features,
    train_targets_nonscored,
) = load_data()

mlp_tf.train_and_evaluate(
    train, test, train_targets, features, train_targets_nonscored, model_type="rs"
)
mlp_tf.inference(
    train, test, train_targets, features, train_targets_nonscored, model_type="rs"
)

fold:5, epochs:80, batch_size:128, LR:0.001
Using model rs with seed 123 for train_and_evaluate
Trained with 874 features. train.shape: (23814, 876). train_targets.shape: (23814, 207)


100%|███████████████████████████████████████████████████████████████████████████████████| 5/5 [01:45<00:00, 21.02s/it]


Our out of folds mean log loss score is 0.01676109207541012
Our out of folds log loss for our seed blend model is 0.01676109207541012
Using model rs with seed 123 for inference
Trained with 874 features


100%|███████████████████████████████████████████████████████████████████████████████████| 5/5 [00:06<00:00,  1.34s/it]


Our out of folds mean log loss score is 0.01676109207541012
Our out of folds log loss for our seed blend model is 0.01676109207541012


(array([[0.00169588, 0.00301255, 0.00215034, ..., 0.00222579, 0.00925308,
         0.00209847],
        [0.0005218 , 0.00137913, 0.00165742, ..., 0.00175286, 0.00656317,
         0.00177591],
        [0.00077693, 0.0006134 , 0.00176651, ..., 0.0020643 , 0.00273696,
         0.00478974],
        ...,
        [0.00195807, 0.00146734, 0.00178714, ..., 0.00204957, 0.00109019,
         0.00175179],
        [0.00252073, 0.00152682, 0.00280283, ..., 0.0023976 , 0.00094649,
         0.00412406],
        [0.00153159, 0.00166212, 0.00198604, ..., 0.00209821, 0.00152105,
         0.00184435]]),
 array([[1.25059416e-03, 2.20698258e-03, 1.21258327e-03, ...,
         9.98972915e-04, 7.55606510e-04, 4.94353892e-03],
        [6.50121691e-03, 3.05776577e-03, 2.63947388e-03, ...,
         2.59770895e-03, 9.03046783e-03, 5.95065486e-03],
        [3.74499732e-03, 2.58676708e-03, 1.75386970e-03, ...,
         2.30530417e-03, 1.51384482e-03, 1.12211134e-03],
        ...,
        [1.22588500e-03, 1.47365592e

In [14]:
str_condition = "no feature_eng"
(
    train,
    train_targets,
    test,
    sample_submission,
    features,
    train_targets_nonscored,
) = load_data()
# モデルブレンドの学習実行できるかテスト
mlp_tf.run_mlp_tf_blend_logger(
    train,
    test,
    train_targets,
    features,
    train_targets_nonscored,
    str_condition=str_condition,
)
# モデルブレンドの推論実行できるかテスト
mlp_tf.run_mlp_tf_blend_logger(
    train,
    test,
    train_targets,
    features,
    train_targets_nonscored,
    str_condition=str_condition,
    is_train=False,
)

fold:5, epochs:80, batch_size:128, LR:0.001
Using model lr with seed 123 for train_and_evaluate
Trained with 874 features. train.shape: (23814, 876). train_targets.shape: (23814, 207)


100%|███████████████████████████████████████████████████████████████████████████████████| 5/5 [01:33<00:00, 18.75s/it]


Our out of folds mean log loss score is 0.020149788824180583
Our out of folds log loss for our seed blend model is 0.020149788824180583


[2020-11-05 23:07:07] - model_type:lr, oof:0.0201498, train_flag:train
lr	no feature_eng	0.0201498	train


fold:5, epochs:80, batch_size:128, LR:0.001
Using model 2l with seed 123 for train_and_evaluate
Trained with 874 features. train.shape: (23814, 876). train_targets.shape: (23814, 207)


...

 80%|██████████████████████████████████████████████████████████████████▍                | 4/5 [00:19<00:04,  4.82s/it]

params: None
Please check your arguments if you have upgraded adabelief-tf from version 0.0.1.
Modifications to default arguments:
                           eps  weight_decouple    rectify
-----------------------  -----  -----------------  -------------
adabelief-tf=0.0.1       1e-08  Not supported      Not supported
Current version (0.1.0)  1e-14  supported          default: True
For a complete table of recommended hyperparameters, see
https://github.com/juntang-zhuang/Adabelief-Optimizer



100%|███████████████████████████████████████████████████████████████████████████████████| 5/5 [00:24<00:00,  4.82s/it]


Our out of folds mean log loss score is 0.016926636221588593
Our out of folds log loss for our seed blend model is 0.016926636221588593


[2020-11-05 23:41:35] - model_type:tabnet_class, oof:0.0169266, train_flag:inference
tabnet_class	no feature_eng	0.0169266	inference
[2020-11-05 23:41:39] - model_type:lr-2l-3l-4l-5l-rs-3l_v2-3lWN-stack_tabnet-tabnet_class, oof:0.0207735, train_flag:inference
lr-2l-3l-4l-5l-rs-3l_v2-3lWN-stack_tabnet-tabnet_class	no feature_eng:Mean blend	0.0207735	inference


## best FE?

In [15]:
str_condition = (
    "fe_stats flag_add=False → c_squared → c_abs → g_valid → fe_pca → scaling"
)
(
    train,
    train_targets,
    test,
    sample_submission,
    features,
    train_targets_nonscored,
) = load_data()
train, test, features = datasets.fe_stats(train, test, flag_add=False)
train, test, features = datasets.c_squared(train, test)
train, test, features = datasets.c_abs(train, test)
train, test, features = datasets.g_valid(train, test)
train, test, features = datasets.fe_pca(
    train, test, n_components_g=70, n_components_c=10, SEED=123
)
train, test, features = datasets.scaling(train, test)
mlp_tf.run_mlp_tf_blend_logger(
    train,
    test,
    train_targets,
    features,
    train_targets_nonscored,
    str_condition=str_condition,
)

fold:5, epochs:80, batch_size:128, LR:0.001
Using model lr with seed 123 for train_and_evaluate
Trained with 1941 features. train.shape: (23814, 1943). train_targets.shape: (23814, 207)


100%|███████████████████████████████████████████████████████████████████████████████████| 5/5 [02:50<00:00, 34.01s/it]


Our out of folds mean log loss score is 0.018652729418941142
Our out of folds log loss for our seed blend model is 0.018652729418941142


[2020-11-05 23:45:38] - model_type:lr, oof:0.0186527, train_flag:train
lr	fe_stats flag_add=False → c_squared → c_abs → g_valid → fe_pca → scaling	0.0186527	train


fold:5, epochs:80, batch_size:128, LR:0.001
Using model 2l with seed 123 for train_and_evaluate
Trained with 1941 features. train.shape: (23814, 1943). train_targets.shape: (23814, 207)


...

 80%|██████████████████████████████████████████████████████████████████▍                | 4/5 [01:34<00:23, 23.35s/it]

params: None
Please check your arguments if you have upgraded adabelief-tf from version 0.0.1.
Modifications to default arguments:
                           eps  weight_decouple    rectify
-----------------------  -----  -----------------  -------------
adabelief-tf=0.0.1       1e-08  Not supported      Not supported
Current version (0.1.0)  1e-14  supported          default: True
For a complete table of recommended hyperparameters, see
https://github.com/juntang-zhuang/Adabelief-Optimizer



100%|███████████████████████████████████████████████████████████████████████████████████| 5/5 [01:59<00:00, 23.81s/it]


Our out of folds mean log loss score is 0.017277233621319964
Our out of folds log loss for our seed blend model is 0.017277233621319964


[2020-11-06 00:17:36] - model_type:tabnet_class, oof:0.0172772, train_flag:train
tabnet_class	fe_stats flag_add=False → c_squared → c_abs → g_valid → fe_pca → scaling	0.0172772	train
[2020-11-06 00:17:39] - model_type:lr-2l-3l-4l-5l-rs-3l_v2-3lWN-stack_tabnet-tabnet_class, oof:0.0208002, train_flag:train
lr-2l-3l-4l-5l-rs-3l_v2-3lWN-stack_tabnet-tabnet_class	fe_stats flag_add=False → c_squared → c_abs → g_valid → fe_pca → scaling:Mean blend	0.0208002	train


In [41]:
str_condition = (
    "fe_stats flag_add=False → c_squared → c_abs → g_valid → fe_pca → scaling → fe_clipping"
)
(
    train,
    train_targets,
    test,
    sample_submission,
    features,
    train_targets_nonscored,
) = load_data()
train, test, features = datasets.fe_stats(train, test, flag_add=False)
train, test, features = datasets.c_squared(train, test)
train, test, features = datasets.c_abs(train, test)
train, test, features = datasets.g_valid(train, test)
train, test, features = datasets.fe_pca(
    train, test, n_components_g=70, n_components_c=10, SEED=123
)
train, test, features = datasets.scaling(train, test)
features_g, features_c = datasets.get_features_gc(train)
train, test = datasets.fe_clipping(train, test, features_g=features_g, features_c=features_c)
mlp_tf.run_mlp_tf_blend_logger(
    train,
    test,
    train_targets,
    features,
    train_targets_nonscored,
    model_types=[
        "2l",
        "3l",
        "4l",
        "5l",
        "rs",
        "3l_v2",
        "stack_tabnet",
        "tabnet_class",
    ],
    str_condition=str_condition,
)

1.0 / 99.0 % clip : -2.566718995290424 / 2.5704835731583646
1.0 / 99.0 % clip : -6.858598915666221 / 65.34177977047811
fold:5, epochs:80, batch_size:128, LR:0.001
Using model 2l with seed 123 for train_and_evaluate
Trained with 1941 features. train.shape: (23814, 1943). train_targets.shape: (23814, 207)


100%|███████████████████████████████████████████████████████████████████████████████████| 5/5 [06:07<00:00, 73.42s/it]


Our out of folds mean log loss score is 0.016288235437457776
Our out of folds log loss for our seed blend model is 0.016288235437457776


[2020-11-06 11:43:31] - model_type:2l, oof:0.0162882, train_flag:train
2l	fe_stats flag_add=False → c_squared → c_abs → g_valid → fe_pca → scaling → fe_clipping	0.0162882	train


fold:5, epochs:80, batch_size:128, LR:0.001
Using model 3l with seed 123 for train_and_evaluate
Trained with 1941 features. train.shape: (23814, 1943). train_targets.shape: (23814, 207)


...

 80%|██████████████████████████████████████████████████████████████████▍                | 4/5 [01:28<00:22, 22.00s/it]

params: None
Please check your arguments if you have upgraded adabelief-tf from version 0.0.1.
Modifications to default arguments:
                           eps  weight_decouple    rectify
-----------------------  -----  -----------------  -------------
adabelief-tf=0.0.1       1e-08  Not supported      Not supported
Current version (0.1.0)  1e-14  supported          default: True
For a complete table of recommended hyperparameters, see
https://github.com/juntang-zhuang/Adabelief-Optimizer



100%|███████████████████████████████████████████████████████████████████████████████████| 5/5 [01:50<00:00, 22.08s/it]


Our out of folds mean log loss score is 0.017294507869808524
Our out of folds log loss for our seed blend model is 0.017294507869808524


[2020-11-06 12:06:35] - model_type:tabnet_class, oof:0.0172945, train_flag:train
tabnet_class	fe_stats flag_add=False → c_squared → c_abs → g_valid → fe_pca → scaling → fe_clipping	0.0172945	train
[2020-11-06 12:06:38] - model_type:2l-3l-4l-5l-rs-3l_v2-stack_tabnet-tabnet_class, oof:0.020213, train_flag:train
2l-3l-4l-5l-rs-3l_v2-stack_tabnet-tabnet_class	fe_stats flag_add=False → c_squared → c_abs → g_valid → fe_pca → scaling → fe_clipping:Mean blend	0.020213	train


In [17]:
str_condition = "fe_stats flag_add=False → c_squared → c_abs → g_valid → fe_pca → scaling + seeds=[5, 12, 67]"
(
    train,
    train_targets,
    test,
    sample_submission,
    features,
    train_targets_nonscored,
) = load_data()
train, test, features = datasets.fe_stats(train, test, flag_add=False)
train, test, features = datasets.c_squared(train, test)
train, test, features = datasets.c_abs(train, test)
train, test, features = datasets.g_valid(train, test)
train, test, features = datasets.fe_pca(
    train, test, n_components_g=70, n_components_c=10, SEED=123
)
train, test, features = datasets.scaling(train, test)
mlp_tf.run_mlp_tf_blend_logger(
    train,
    test,
    train_targets,
    features,
    train_targets_nonscored,
    seeds=[5, 12, 67],
    str_condition=str_condition,
    model_types=["rs", "2l", "4l", "5l", "3l", "stack_tabnet"],
)

fold:5, epochs:80, batch_size:128, LR:0.001
Using model rs with seed 5 for train_and_evaluate
Trained with 1941 features. train.shape: (23814, 1943). train_targets.shape: (23814, 207)


100%|███████████████████████████████████████████████████████████████████████████████████| 5/5 [02:05<00:00, 25.14s/it]


Our out of folds mean log loss score is 0.016736725817493185
Using model rs with seed 12 for train_and_evaluate
Trained with 1941 features. train.shape: (23814, 1943). train_targets.shape: (23814, 206)


100%|███████████████████████████████████████████████████████████████████████████████████| 5/5 [01:42<00:00, 20.50s/it]


Our out of folds mean log loss score is 0.01672545993069803
Using model rs with seed 67 for train_and_evaluate
Trained with 1941 features. train.shape: (23814, 1943). train_targets.shape: (23814, 206)


...

 80%|██████████████████████████████████████████████████████████████████▍                | 4/5 [02:19<00:34, 34.86s/it]

params: None
Please check your arguments if you have upgraded adabelief-tf from version 0.0.1.
Modifications to default arguments:
                           eps  weight_decouple    rectify
-----------------------  -----  -----------------  -------------
adabelief-tf=0.0.1       1e-08  Not supported      Not supported
Current version (0.1.0)  1e-14  supported          default: True
For a complete table of recommended hyperparameters, see
https://github.com/juntang-zhuang/Adabelief-Optimizer



100%|███████████████████████████████████████████████████████████████████████████████████| 5/5 [02:55<00:00, 35.10s/it]


Our out of folds mean log loss score is 0.017434247206467173
Our out of folds log loss for our seed blend model is 0.016998437133547813


[2020-11-06 01:40:18] - model_type:stack_tabnet, oof:0.0169984, train_flag:train
stack_tabnet	fe_stats flag_add=False → c_squared → c_abs → g_valid → fe_pca → scaling + seeds=[5, 12, 67]	0.0169984	train
[2020-11-06 01:40:21] - model_type:rs-2l-4l-5l-3l-stack_tabnet, oof:0.0192757, train_flag:train
rs-2l-4l-5l-3l-stack_tabnet	fe_stats flag_add=False → c_squared → c_abs → g_valid → fe_pca → scaling + seeds=[5, 12, 67]:Mean blend	0.0192757	train


## 1モデルだけで試していく

In [38]:
str_condition = (
    "fe_clipping → fe_stats flag_add=False → c_squared → c_abs → g_valid → fe_pca → scaling"
)
(
    train,
    train_targets,
    test,
    sample_submission,
    features,
    train_targets_nonscored,
) = load_data()
train, test = datasets.fe_clipping(train, test)
train, test, features = datasets.fe_stats(train, test, flag_add=False)
train, test, features = datasets.c_squared(train, test)
train, test, features = datasets.c_abs(train, test)
train, test, features = datasets.g_valid(train, test)
train, test, features = datasets.fe_pca(
    train, test, n_components_g=70, n_components_c=10, SEED=123
)
train, test, features = datasets.scaling(train, test)
_, _ = mlp_tf.run_mlp_tf_logger(
    train,
    test,
    train_targets,
    features,
    train_targets_nonscored,
    model_type="4l",
    str_condition=str_condition,
)

1.0 / 99.0 % clip : -3.97 / 3.918
1.0 / 99.0 % clip : -9.931 / 1.505
fold:5, epochs:80, batch_size:128, LR:0.001
Using model 4l with seed 123 for train_and_evaluate
Trained with 1941 features. train.shape: (23814, 1943). train_targets.shape: (23814, 207)


100%|███████████████████████████████████████████████████████████████████████████████████| 5/5 [02:55<00:00, 35.16s/it]


Our out of folds mean log loss score is 0.016383323505001532
Our out of folds log loss for our seed blend model is 0.016383323505001532


[2020-11-06 11:25:31] - model_type:4l, oof:0.0163833, train_flag:train
4l	fe_clipping → fe_stats flag_add=False → c_squared → c_abs → g_valid → fe_pca → scaling	0.0163833	train


In [40]:
str_condition = (
    "fe_stats flag_add=False → c_squared → c_abs → g_valid → fe_pca → scaling → fe_clipping"
)
(
    train,
    train_targets,
    test,
    sample_submission,
    features,
    train_targets_nonscored,
) = load_data()
train, test, features = datasets.fe_stats(train, test, flag_add=False)
train, test, features = datasets.c_squared(train, test)
train, test, features = datasets.c_abs(train, test)
train, test, features = datasets.g_valid(train, test)
train, test, features = datasets.fe_pca(
    train, test, n_components_g=70, n_components_c=10, SEED=123
)
train, test, features = datasets.scaling(train, test)
features_g, features_c = datasets.get_features_gc(train)
train, test = datasets.fe_clipping(train, test, features_g=features_g, features_c=features_c)
_, _ = mlp_tf.run_mlp_tf_logger(
    train,
    test,
    train_targets,
    features,
    train_targets_nonscored,
    model_type="4l",
    str_condition=str_condition,
)

1.0 / 99.0 % clip : -2.566718995290424 / 2.5704835731583646
1.0 / 99.0 % clip : -6.858598915666221 / 65.34177977047811
fold:5, epochs:80, batch_size:128, LR:0.001
Using model 4l with seed 123 for train_and_evaluate
Trained with 1941 features. train.shape: (23814, 1943). train_targets.shape: (23814, 207)


100%|███████████████████████████████████████████████████████████████████████████████████| 5/5 [02:48<00:00, 33.65s/it]


Our out of folds mean log loss score is 0.016353718597731066
Our out of folds log loss for our seed blend model is 0.016353718597731066


[2020-11-06 11:32:48] - model_type:4l, oof:0.0163537, train_flag:train
4l	fe_stats flag_add=False → c_squared → c_abs → g_valid → fe_pca → scaling → fe_clipping	0.0163537	train


In [54]:
str_condition = (
    "no feature_eng + is_del_ctl + is_del_noise_drug"
)
(
    train,
    train_targets,
    test,
    sample_submission,
    train_targets_nonscored,
    train_drug,
) = datasets.load_orig_data()
train, train_targets, test, train_targets_nonscored = datasets.mapping_and_filter(
    train, train_targets, test, train_targets_nonscored, is_del_ctl=True, is_del_noise_drug=True
)
features = datasets.get_features(train)

_, _ = mlp_tf.run_mlp_tf_logger(
    train,
    test,
    train_targets,
    features,
    train_targets_nonscored,
    model_type="4l",
    str_condition=str_condition,
)

fold:5, epochs:80, batch_size:128, LR:0.001
Using model 4l with seed 123 for train_and_evaluate
Trained with 874 features. train.shape: (21282, 876). train_targets.shape: (21282, 207)


100%|███████████████████████████████████████████████████████████████████████████████████| 5/5 [02:36<00:00, 31.40s/it]


Our out of folds mean log loss score is 0.0175857142120379
Our out of folds log loss for our seed blend model is 0.0175857142120379


[2020-11-06 09:11:25] - model_type:4l, oof:0.0175857, train_flag:train
4l	no feature_eng + is_del_ctl + is_del_noise_drug	0.0175857	train


In [11]:
str_condition = (
    "is_del_ctl + scaling"
)
(
    train,
    train_targets,
    test,
    sample_submission,
    train_targets_nonscored,
    train_drug,
) = datasets.load_orig_data()
train, train_targets, test, train_targets_nonscored = datasets.mapping_and_filter(
    train, train_targets, test, train_targets_nonscored, is_del_ctl=True
)
train, test, features = datasets.scaling(train, test)
_, _ = mlp_tf.run_mlp_tf_logger(
    train,
    test,
    train_targets,
    features,
    train_targets_nonscored,
    model_type="4l",
    str_condition=str_condition,
)

fold:5, epochs:80, batch_size:128, LR:0.001
Using model 4l with seed 123 for train_and_evaluate
Trained with 874 features. train.shape: (21948, 876). train_targets.shape: (21948, 207)


100%|███████████████████████████████████████████████████████████████████████████████████| 5/5 [02:15<00:00, 27.16s/it]


Our out of folds mean log loss score is 0.017734774967382398
Our out of folds log loss for our seed blend model is 0.017734774967382398


[2020-11-05 22:59:54] - model_type:4l, oof:0.0177348, train_flag:train
4l	is_del_ctl + scaling	0.0177348	train


In [13]:
str_condition = (
    "is_del_noise_drug + scaling"
)
(
    train,
    train_targets,
    test,
    sample_submission,
    train_targets_nonscored,
    train_drug,
) = datasets.load_orig_data()
train, train_targets, test, train_targets_nonscored = datasets.mapping_and_filter(
    train, train_targets, test, train_targets_nonscored, is_del_noise_drug=True
)
train, test, features = datasets.scaling(train, test)
_, _ = mlp_tf.run_mlp_tf_logger(
    train,
    test,
    train_targets,
    features,
    train_targets_nonscored,
    model_type="4l",
    str_condition=str_condition,
)

fold:5, epochs:80, batch_size:128, LR:0.001
Using model 4l with seed 123 for train_and_evaluate
Trained with 874 features. train.shape: (23096, 876). train_targets.shape: (23096, 207)


100%|███████████████████████████████████████████████████████████████████████████████████| 5/5 [02:31<00:00, 30.29s/it]


Our out of folds mean log loss score is 0.016999163032590504
Our out of folds log loss for our seed blend model is 0.016999163032590504


[2020-11-05 23:05:18] - model_type:4l, oof:0.0169992, train_flag:train
4l	is_del_noise_drug + scaling	0.0169992	train


In [12]:
str_condition = (
    "is_conat_nonscore + scaling"
)
(
    train,
    train_targets,
    test,
    sample_submission,
    train_targets_nonscored,
    train_drug,
) = datasets.load_orig_data()
train, train_targets, test, train_targets_nonscored = datasets.mapping_and_filter(
    train, train_targets, test, train_targets_nonscored, is_conat_nonscore=True
)
train, test, features = datasets.scaling(train, test)
_, _ = mlp_tf.run_mlp_tf_logger(
    train,
    test,
    train_targets,
    features,
    train_targets_nonscored,
    model_type="4l",
    str_condition=str_condition,
)

fold:5, epochs:80, batch_size:128, LR:0.001
Using model 4l with seed 123 for train_and_evaluate
Trained with 874 features. train.shape: (23814, 876). train_targets.shape: (23814, 538)


100%|███████████████████████████████████████████████████████████████████████████████████| 5/5 [02:21<00:00, 28.32s/it]


Our out of folds mean log loss score is 0.016740740054203725
Our out of folds log loss for our seed blend model is 0.016740740054203725


[2020-11-05 23:02:35] - model_type:4l, oof:0.0167407, train_flag:train
4l	is_conat_nonscore + scaling	0.0167407	train


In [18]:
for bs in [8, 16, 32, 64, 128, 256, 1024]:
    mlp_tf.BATCH_SIZE = bs
    str_condition = f"no feature_eng + batch_size={mlp_tf.BATCH_SIZE}"
    (
        train,
        train_targets,
        test,
        sample_submission,
        features,
        train_targets_nonscored,
    ) = load_data()
    _, _ = mlp_tf.run_mlp_tf_logger(
        train,
        test,
        train_targets,
        features,
        train_targets_nonscored,
        model_type="tabnet_class",
        str_condition=str_condition,
    )
    (
        train,
        train_targets,
        test,
        sample_submission,
        features,
        train_targets_nonscored,
    ) = load_data()
    _, _ = mlp_tf.run_mlp_tf_logger(
        train,
        test,
        train_targets,
        features,
        train_targets_nonscored,
        model_type="stack_tabnet",
        str_condition=str_condition,
    )
    
mlp_tf.BATCH_SIZE = 128

fold:5, epochs:80, batch_size:8, LR:0.001
Using model tabnet_class with seed 123 for train_and_evaluate
Trained with 874 features. train.shape: (23814, 876). train_targets.shape: (23814, 207)


  0%|                                                                                           | 0/5 [00:00<?, ?it/s]

params: None
Please check your arguments if you have upgraded adabelief-tf from version 0.0.1.
Modifications to default arguments:
                           eps  weight_decouple    rectify
-----------------------  -----  -----------------  -------------
adabelief-tf=0.0.1       1e-08  Not supported      Not supported
Current version (0.1.0)  1e-14  supported          default: True
For a complete table of recommended hyperparameters, see
https://github.com/juntang-zhuang/Adabelief-Optimizer



 20%|████████████████▍                                                                 | 1/5 [03:14<12:59, 194.86s/it]

params: None
Please check your arguments if you have upgraded adabelief-tf from version 0.0.1.
Modifications to default arguments:
                           eps  weight_decouple    rectify
-----------------------  -----  -----------------  -------------
adabelief-tf=0.0.1       1e-08  Not supported      Not supported
Current version (0.1.0)  1e-14  supported          default: True
For a complete table of recommended hyperparameters, see
https://github.com/juntang-zhuang/Adabelief-Optimizer



...

 80%|██████████████████████████████████████████████████████████████████▍                | 4/5 [01:29<00:22, 22.89s/it]

params: None
Please check your arguments if you have upgraded adabelief-tf from version 0.0.1.
Modifications to default arguments:
                           eps  weight_decouple    rectify
-----------------------  -----  -----------------  -------------
adabelief-tf=0.0.1       1e-08  Not supported      Not supported
Current version (0.1.0)  1e-14  supported          default: True
For a complete table of recommended hyperparameters, see
https://github.com/juntang-zhuang/Adabelief-Optimizer



100%|███████████████████████████████████████████████████████████████████████████████████| 5/5 [01:50<00:00, 22.16s/it]


Our out of folds mean log loss score is 0.017493571929670482
Our out of folds log loss for our seed blend model is 0.017493571929670482


[2020-11-06 03:27:04] - model_type:stack_tabnet, oof:0.0174936, train_flag:train
stack_tabnet	no feature_eng + batch_size=1024	0.0174936	train


In [19]:
for lr in [0.1, 0.01, 0.0001, 0.00001]:
    mlp_tf.LR = lr
    str_condition = f"no feature_eng + learning rate={mlp_tf.LR}"
    (
        train,
        train_targets,
        test,
        sample_submission,
        features,
        train_targets_nonscored,
    ) = load_data()
    _, _ = mlp_tf.run_mlp_tf_logger(
        train,
        test,
        train_targets,
        features,
        train_targets_nonscored,
        model_type="tabnet_class",
        str_condition=str_condition,
    )
    (
        train,
        train_targets,
        test,
        sample_submission,
        features,
        train_targets_nonscored,
    ) = load_data()
    _, _ = mlp_tf.run_mlp_tf_logger(
        train,
        test,
        train_targets,
        features,
        train_targets_nonscored,
        model_type="stack_tabnet",
        str_condition=str_condition,
    )
    
mlp_tf.LR = 0.001

fold:5, epochs:80, batch_size:128, LR:0.1
Using model tabnet_class with seed 123 for train_and_evaluate
Trained with 874 features. train.shape: (23814, 876). train_targets.shape: (23814, 207)


  0%|                                                                                           | 0/5 [00:00<?, ?it/s]

params: None
Please check your arguments if you have upgraded adabelief-tf from version 0.0.1.
Modifications to default arguments:
                           eps  weight_decouple    rectify
-----------------------  -----  -----------------  -------------
adabelief-tf=0.0.1       1e-08  Not supported      Not supported
Current version (0.1.0)  1e-14  supported          default: True
For a complete table of recommended hyperparameters, see
https://github.com/juntang-zhuang/Adabelief-Optimizer



 20%|████████████████▌                                                                  | 1/5 [00:20<01:22, 20.61s/it]

params: None
Please check your arguments if you have upgraded adabelief-tf from version 0.0.1.
Modifications to default arguments:
                           eps  weight_decouple    rectify
-----------------------  -----  -----------------  -------------
adabelief-tf=0.0.1       1e-08  Not supported      Not supported
Current version (0.1.0)  1e-14  supported          default: True
For a complete table of recommended hyperparameters, see
https://github.com/juntang-zhuang/Adabelief-Optimizer



...

 80%|█████████████████████████████████████████████████████████████████▌                | 4/5 [08:48<02:11, 131.73s/it]

params: None
Please check your arguments if you have upgraded adabelief-tf from version 0.0.1.
Modifications to default arguments:
                           eps  weight_decouple    rectify
-----------------------  -----  -----------------  -------------
adabelief-tf=0.0.1       1e-08  Not supported      Not supported
Current version (0.1.0)  1e-14  supported          default: True
For a complete table of recommended hyperparameters, see
https://github.com/juntang-zhuang/Adabelief-Optimizer



100%|██████████████████████████████████████████████████████████████████████████████████| 5/5 [11:03<00:00, 132.72s/it]


Our out of folds mean log loss score is 0.01869354108160658
Our out of folds log loss for our seed blend model is 0.01869354108160658


[2020-11-06 04:05:14] - model_type:stack_tabnet, oof:0.0186935, train_flag:train
stack_tabnet	no feature_eng + learning rate=1e-05	0.0186935	train


In [20]:
for num_decision_steps in [1, 2, 3, 4, 8]:
    str_condition = f"no feature_eng + num_decision_steps={num_decision_steps}"
    (
        train,
        train_targets,
        test,
        sample_submission,
        features,
        train_targets_nonscored,
    ) = load_data()
    tabnet_params = dict(
            feature_columns=None,
            num_classes=206,
            feature_dim=128,
            output_dim=64,
            num_features=len(features),
            num_decision_steps=num_decision_steps,
            relaxation_factor=1.5,
            sparsity_coefficient=1e-5,
            batch_momentum=0.98,
            virtual_batch_size=None,
            norm_type="group",
            num_groups=-1,
            multi_label=True,
        )
    _, _ = mlp_tf.run_mlp_tf_logger(
        train,
        test,
        train_targets,
        features,
        train_targets_nonscored,
        model_type="tabnet_class",
        str_condition=str_condition,
        tabnet_params=tabnet_params,
    )
    (
        train,
        train_targets,
        test,
        sample_submission,
        features,
        train_targets_nonscored,
    ) = load_data()
    tabnet_params = dict(
            feature_columns=None,
            num_layers=2,
            num_classes=206,
            feature_dim=128,
            output_dim=64,
            num_features=len(features),
            num_decision_steps=num_decision_steps,
            relaxation_factor=1.5,
            sparsity_coefficient=1e-5,
            batch_momentum=0.98,
            virtual_batch_size=None,
            norm_type="group",
            num_groups=-1,
            multi_label=True,
        )
    _, _ = mlp_tf.run_mlp_tf_logger(
        train,
        test,
        train_targets,
        features,
        train_targets_nonscored,
        model_type="stack_tabnet",
        str_condition=str_condition,
        tabnet_params=tabnet_params,
    )

fold:5, epochs:80, batch_size:128, LR:0.001
Using model tabnet_class with seed 123 for train_and_evaluate
Trained with 874 features. train.shape: (23814, 876). train_targets.shape: (23814, 207)


  0%|                                                                                           | 0/5 [00:00<?, ?it/s]

params: {'feature_columns': None, 'num_classes': 206, 'feature_dim': 128, 'output_dim': 64, 'num_features': 874, 'num_decision_steps': 1, 'relaxation_factor': 1.5, 'sparsity_coefficient': 1e-05, 'batch_momentum': 0.98, 'virtual_batch_size': None, 'norm_type': 'group', 'num_groups': -1, 'multi_label': True}
Please check your arguments if you have upgraded adabelief-tf from version 0.0.1.
Modifications to default arguments:
                           eps  weight_decouple    rectify
-----------------------  -----  -----------------  -------------
adabelief-tf=0.0.1       1e-08  Not supported      Not supported
Current version (0.1.0)  1e-14  supported          default: True
For a complete table of recommended hyperparameters, see
https://github.com/juntang-zhuang/Adabelief-Optimizer



 20%|████████████████▌                                                                  | 1/5 [00:25<01:41, 25.46s/it]

params: {'feature_columns': None, 'num_classes': 206, 'feature_dim': 128, 'output_dim': 64, 'num_features': 874, 'num_decision_steps': 1, 'relaxation_factor': 1.5, 'sparsity_coefficient': 1e-05, 'batch_momentum': 0.98, 'virtual_batch_size': None, 'norm_type': 'group', 'num_groups': -1, 'multi_label': True}
Please check your arguments if you have upgraded adabelief-tf from version 0.0.1.
Modifications to default arguments:
                           eps  weight_decouple    rectify
-----------------------  -----  -----------------  -------------
adabelief-tf=0.0.1       1e-08  Not supported      Not supported
Current version (0.1.0)  1e-14  supported          default: True
For a complete table of recommended hyperparameters, see
https://github.com/juntang-zhuang/Adabelief-Optimizer



...

 80%|█████████████████████████████████████████████████████████████████▌                | 4/5 [32:46<07:43, 463.95s/it]

params: {'feature_columns': None, 'num_layers': 2, 'num_classes': 206, 'feature_dim': 128, 'output_dim': 64, 'num_features': 874, 'num_decision_steps': 8, 'relaxation_factor': 1.5, 'sparsity_coefficient': 1e-05, 'batch_momentum': 0.98, 'virtual_batch_size': None, 'norm_type': 'group', 'num_groups': -1, 'multi_label': True}
Please check your arguments if you have upgraded adabelief-tf from version 0.0.1.
Modifications to default arguments:
                           eps  weight_decouple    rectify
-----------------------  -----  -----------------  -------------
adabelief-tf=0.0.1       1e-08  Not supported      Not supported
Current version (0.1.0)  1e-14  supported          default: True
For a complete table of recommended hyperparameters, see
https://github.com/juntang-zhuang/Adabelief-Optimizer



100%|██████████████████████████████████████████████████████████████████████████████████| 5/5 [44:03<00:00, 528.79s/it]


Our out of folds mean log loss score is 0.02057137782154323
Our out of folds log loss for our seed blend model is 0.02057137782154323


[2020-11-06 06:37:15] - model_type:stack_tabnet, oof:0.0205714, train_flag:train
stack_tabnet	no feature_eng + num_decision_steps=8	0.0205714	train


In [21]:
for feature_dim in [8, 32, 64, 128]:
    str_condition = f"no feature_eng + feature_dim,output_dim={feature_dim}"
    (
        train,
        train_targets,
        test,
        sample_submission,
        features,
        train_targets_nonscored,
    ) = load_data()
    tabnet_params = dict(
            feature_columns=None,
            num_classes=206,
            feature_dim=feature_dim,
            output_dim=feature_dim,
            num_features=len(features),
            num_decision_steps=1,
            relaxation_factor=1.5,
            sparsity_coefficient=1e-5,
            batch_momentum=0.98,
            virtual_batch_size=None,
            norm_type="group",
            num_groups=-1,
            multi_label=True,
        )
    _, _ = mlp_tf.run_mlp_tf_logger(
        train,
        test,
        train_targets,
        features,
        train_targets_nonscored,
        model_type="tabnet_class",
        str_condition=str_condition,
        tabnet_params=tabnet_params,
    )
    (
        train,
        train_targets,
        test,
        sample_submission,
        features,
        train_targets_nonscored,
    ) = load_data()
    tabnet_params = dict(
            feature_columns=None,
            num_classes=206,
            num_layers=2,
            feature_dim=feature_dim,
            output_dim=feature_dim,
            num_features=len(features),
            num_decision_steps=1,
            relaxation_factor=1.5,
            sparsity_coefficient=1e-5,
            batch_momentum=0.98,
            virtual_batch_size=None,
            norm_type="group",
            num_groups=-1,
            multi_label=True,
        )
    _, _ = mlp_tf.run_mlp_tf_logger(
        train,
        test,
        train_targets,
        features,
        train_targets_nonscored,
        model_type="stack_tabnet",
        str_condition=str_condition,
        tabnet_params=tabnet_params,
    )

fold:5, epochs:80, batch_size:128, LR:0.001
Using model tabnet_class with seed 123 for train_and_evaluate
Trained with 874 features. train.shape: (23814, 876). train_targets.shape: (23814, 207)


  0%|                                                                                           | 0/5 [00:00<?, ?it/s]

params: {'feature_columns': None, 'num_classes': 206, 'feature_dim': 8, 'output_dim': 8, 'num_features': 874, 'num_decision_steps': 1, 'relaxation_factor': 1.5, 'sparsity_coefficient': 1e-05, 'batch_momentum': 0.98, 'virtual_batch_size': None, 'norm_type': 'group', 'num_groups': -1, 'multi_label': True}
Please check your arguments if you have upgraded adabelief-tf from version 0.0.1.
Modifications to default arguments:
                           eps  weight_decouple    rectify
-----------------------  -----  -----------------  -------------
adabelief-tf=0.0.1       1e-08  Not supported      Not supported
Current version (0.1.0)  1e-14  supported          default: True
For a complete table of recommended hyperparameters, see
https://github.com/juntang-zhuang/Adabelief-Optimizer



 20%|████████████████▌                                                                  | 1/5 [00:47<03:10, 47.75s/it]

params: {'feature_columns': None, 'num_classes': 206, 'feature_dim': 8, 'output_dim': 8, 'num_features': 874, 'num_decision_steps': 1, 'relaxation_factor': 1.5, 'sparsity_coefficient': 1e-05, 'batch_momentum': 0.98, 'virtual_batch_size': None, 'norm_type': 'group', 'num_groups': -1, 'multi_label': True}
Please check your arguments if you have upgraded adabelief-tf from version 0.0.1.
Modifications to default arguments:
                           eps  weight_decouple    rectify
-----------------------  -----  -----------------  -------------
adabelief-tf=0.0.1       1e-08  Not supported      Not supported
Current version (0.1.0)  1e-14  supported          default: True
For a complete table of recommended hyperparameters, see
https://github.com/juntang-zhuang/Adabelief-Optimizer



...

 80%|██████████████████████████████████████████████████████████████████▍                | 4/5 [02:32<00:38, 38.64s/it]

params: {'feature_columns': None, 'num_classes': 206, 'num_layers': 2, 'feature_dim': 128, 'output_dim': 128, 'num_features': 874, 'num_decision_steps': 1, 'relaxation_factor': 1.5, 'sparsity_coefficient': 1e-05, 'batch_momentum': 0.98, 'virtual_batch_size': None, 'norm_type': 'group', 'num_groups': -1, 'multi_label': True}
Please check your arguments if you have upgraded adabelief-tf from version 0.0.1.
Modifications to default arguments:
                           eps  weight_decouple    rectify
-----------------------  -----  -----------------  -------------
adabelief-tf=0.0.1       1e-08  Not supported      Not supported
Current version (0.1.0)  1e-14  supported          default: True
For a complete table of recommended hyperparameters, see
https://github.com/juntang-zhuang/Adabelief-Optimizer



100%|███████████████████████████████████████████████████████████████████████████████████| 5/5 [03:09<00:00, 37.90s/it]


Our out of folds mean log loss score is 0.016969490429787654
Our out of folds log loss for our seed blend model is 0.016969490429787654


[2020-11-06 07:06:19] - model_type:stack_tabnet, oof:0.0169695, train_flag:train
stack_tabnet	no feature_eng + feature_dim,output_dim=128	0.0169695	train


In [22]:
for num_layers in [1, 2, 3, 4, 5]:
    str_condition = f"no feature_eng + num_layers={num_layers}"
    (
        train,
        train_targets,
        test,
        sample_submission,
        features,
        train_targets_nonscored,
    ) = load_data()
    tabnet_params = dict(
            feature_columns=None,
            num_classes=206,
            num_layers=num_layers,
            feature_dim=128,
            output_dim=64,
            num_features=len(features),
            num_decision_steps=1,
            relaxation_factor=1.5,
            sparsity_coefficient=1e-5,
            batch_momentum=0.98,
            virtual_batch_size=None,
            norm_type="group",
            num_groups=-1,
            multi_label=True,
        )
    _, _ = mlp_tf.run_mlp_tf_logger(
        train,
        test,
        train_targets,
        features,
        train_targets_nonscored,
        model_type="stack_tabnet",
        str_condition=str_condition,
        tabnet_params=tabnet_params,
    )

fold:5, epochs:80, batch_size:128, LR:0.001
Using model stack_tabnet with seed 123 for train_and_evaluate
Trained with 874 features. train.shape: (23814, 876). train_targets.shape: (23814, 207)


  0%|                                                                                           | 0/5 [00:00<?, ?it/s]

params: {'feature_columns': None, 'num_classes': 206, 'num_layers': 1, 'feature_dim': 128, 'output_dim': 64, 'num_features': 874, 'num_decision_steps': 1, 'relaxation_factor': 1.5, 'sparsity_coefficient': 1e-05, 'batch_momentum': 0.98, 'virtual_batch_size': None, 'norm_type': 'group', 'num_groups': -1, 'multi_label': True}
Please check your arguments if you have upgraded adabelief-tf from version 0.0.1.
Modifications to default arguments:
                           eps  weight_decouple    rectify
-----------------------  -----  -----------------  -------------
adabelief-tf=0.0.1       1e-08  Not supported      Not supported
Current version (0.1.0)  1e-14  supported          default: True
For a complete table of recommended hyperparameters, see
https://github.com/juntang-zhuang/Adabelief-Optimizer



 20%|████████████████▌                                                                  | 1/5 [00:25<01:42, 25.63s/it]

params: {'feature_columns': None, 'num_classes': 206, 'num_layers': 1, 'feature_dim': 128, 'output_dim': 64, 'num_features': 874, 'num_decision_steps': 1, 'relaxation_factor': 1.5, 'sparsity_coefficient': 1e-05, 'batch_momentum': 0.98, 'virtual_batch_size': None, 'norm_type': 'group', 'num_groups': -1, 'multi_label': True}
Please check your arguments if you have upgraded adabelief-tf from version 0.0.1.
Modifications to default arguments:
                           eps  weight_decouple    rectify
-----------------------  -----  -----------------  -------------
adabelief-tf=0.0.1       1e-08  Not supported      Not supported
Current version (0.1.0)  1e-14  supported          default: True
For a complete table of recommended hyperparameters, see
https://github.com/juntang-zhuang/Adabelief-Optimizer



...

 80%|██████████████████████████████████████████████████████████████████▍                | 4/5 [06:32<01:38, 98.68s/it]

params: {'feature_columns': None, 'num_classes': 206, 'num_layers': 5, 'feature_dim': 128, 'output_dim': 64, 'num_features': 874, 'num_decision_steps': 1, 'relaxation_factor': 1.5, 'sparsity_coefficient': 1e-05, 'batch_momentum': 0.98, 'virtual_batch_size': None, 'norm_type': 'group', 'num_groups': -1, 'multi_label': True}
Please check your arguments if you have upgraded adabelief-tf from version 0.0.1.
Modifications to default arguments:
                           eps  weight_decouple    rectify
-----------------------  -----  -----------------  -------------
adabelief-tf=0.0.1       1e-08  Not supported      Not supported
Current version (0.1.0)  1e-14  supported          default: True
For a complete table of recommended hyperparameters, see
https://github.com/juntang-zhuang/Adabelief-Optimizer



100%|███████████████████████████████████████████████████████████████████████████████████| 5/5 [08:14<00:00, 98.96s/it]


Our out of folds mean log loss score is 0.0170164395555092
Our out of folds log loss for our seed blend model is 0.0170164395555092


[2020-11-06 07:31:27] - model_type:stack_tabnet, oof:0.0170164, train_flag:train
stack_tabnet	no feature_eng + num_layers=5	0.0170164	train


In [23]:
str_condition = "cliping"
(
    train,
    train_targets,
    test,
    sample_submission,
    features,
    train_targets_nonscored,
) = load_data()
train, test = datasets.fe_clipping(train, test)
_, _ = mlp_tf.run_mlp_tf_logger(
    train,
    test,
    train_targets,
    features,
    train_targets_nonscored,
    model_type="4l",
    str_condition=str_condition,
)

1.0 / 99.0 % clip : -3.97 / 3.918
1.0 / 99.0 % clip : -9.931 / 1.505
fold:5, epochs:80, batch_size:128, LR:0.001
Using model 4l with seed 123 for train_and_evaluate
Trained with 874 features. train.shape: (23814, 876). train_targets.shape: (23814, 207)


100%|███████████████████████████████████████████████████████████████████████████████████| 5/5 [02:30<00:00, 30.06s/it]


Our out of folds mean log loss score is 0.016399909472230658
Our out of folds log loss for our seed blend model is 0.016399909472230658


[2020-11-06 07:34:14] - model_type:4l, oof:0.0163999, train_flag:train
4l	cliping	0.0163999	train


In [58]:
str_condition = "cliping min_clip=0.05, max_clip=0.95"
(
    train,
    train_targets,
    test,
    sample_submission,
    features,
    train_targets_nonscored,
) = load_data()
train, test = datasets.fe_clipping(train, test, min_clip=0.05, max_clip=0.95)
_, _ = mlp_tf.run_mlp_tf_logger(
    train,
    test,
    train_targets,
    features,
    train_targets_nonscored,
    model_type="4l",
    str_condition=str_condition,
)

5.0 / 95.0 % clip : -1.515 / 1.57
5.0 / 95.0 % clip : -3.9920499999999883 / 1.066
fold:5, epochs:80, batch_size:128, LR:0.001
Using model 4l with seed 123 for train_and_evaluate
Trained with 874 features. train.shape: (23814, 876). train_targets.shape: (23814, 207)


100%|███████████████████████████████████████████████████████████████████████████████████| 5/5 [02:27<00:00, 29.47s/it]


Our out of folds mean log loss score is 0.016414002814469142
Our out of folds log loss for our seed blend model is 0.016414002814469142


[2020-11-06 09:18:59] - model_type:4l, oof:0.016414, train_flag:train
4l	cliping min_clip=0.05, max_clip=0.95	0.016414	train


In [59]:
str_condition = "fe_clipping_col"
(
    train,
    train_targets,
    test,
    sample_submission,
    features,
    train_targets_nonscored,
) = load_data()
#display(np.max(train))
train, test = datasets.fe_clipping_col(train, test)
#display(np.max(train))
_, _ = mlp_tf.run_mlp_tf_logger(
    train,
    test,
    train_targets,
    features,
    train_targets_nonscored,
    model_type="4l",
    str_condition=str_condition,
)

fold:5, epochs:80, batch_size:128, LR:0.001
Using model 4l with seed 123 for train_and_evaluate
Trained with 874 features. train.shape: (23814, 876). train_targets.shape: (23814, 207)


100%|███████████████████████████████████████████████████████████████████████████████████| 5/5 [02:43<00:00, 32.60s/it]


Our out of folds mean log loss score is 0.01640379155735001
Our out of folds log loss for our seed blend model is 0.01640379155735001


[2020-11-06 09:22:00] - model_type:4l, oof:0.0164038, train_flag:train
4l	fe_clipping_col	0.0164038	train


In [60]:
str_condition = "fe_clipping_col min_clip=0.05, max_clip=0.95"
(
    train,
    train_targets,
    test,
    sample_submission,
    features,
    train_targets_nonscored,
) = load_data()
#display(np.max(train))
train, test = datasets.fe_clipping_col(train, test, min_clip=0.05, max_clip=0.95)
#display(np.max(train))
_, _ = mlp_tf.run_mlp_tf_logger(
    train,
    test,
    train_targets,
    features,
    train_targets_nonscored,
    model_type="4l",
    str_condition=str_condition,
)

fold:5, epochs:80, batch_size:128, LR:0.001
Using model 4l with seed 123 for train_and_evaluate
Trained with 874 features. train.shape: (23814, 876). train_targets.shape: (23814, 207)


100%|███████████████████████████████████████████████████████████████████████████████████| 5/5 [02:22<00:00, 28.47s/it]


Our out of folds mean log loss score is 0.016424846292118062
Our out of folds log loss for our seed blend model is 0.016424846292118062


[2020-11-06 09:24:41] - model_type:4l, oof:0.0164248, train_flag:train
4l	fe_clipping_col min_clip=0.05, max_clip=0.95	0.0164248	train


In [24]:
str_condition = "cliping → fe_stats"
(
    train,
    train_targets,
    test,
    sample_submission,
    features,
    train_targets_nonscored,
) = load_data()
train, test = datasets.fe_clipping(train, test)
train, test, features = datasets.fe_stats(train, test, flag_add=False)
_, _ = mlp_tf.run_mlp_tf_logger(
    train,
    test,
    train_targets,
    features,
    train_targets_nonscored,
    model_type="4l",
    str_condition=str_condition,
)

1.0 / 99.0 % clip : -3.97 / 3.918
1.0 / 99.0 % clip : -9.931 / 1.505
fold:5, epochs:80, batch_size:128, LR:0.001
Using model 4l with seed 123 for train_and_evaluate
Trained with 889 features. train.shape: (23814, 891). train_targets.shape: (23814, 207)


100%|███████████████████████████████████████████████████████████████████████████████████| 5/5 [02:59<00:00, 35.81s/it]


Our out of folds mean log loss score is 0.016371620847609153
Our out of folds log loss for our seed blend model is 0.016371620847609153


[2020-11-06 07:37:36] - model_type:4l, oof:0.0163716, train_flag:train
4l	cliping → fe_stats	0.0163716	train


In [25]:
str_condition = "cliping → scaling"
(
    train,
    train_targets,
    test,
    sample_submission,
    features,
    train_targets_nonscored,
) = load_data()
train, test = datasets.fe_clipping(train, test)
train, test, features = datasets.scaling(train, test)
_, _ = mlp_tf.run_mlp_tf_logger(
    train,
    test,
    train_targets,
    features,
    train_targets_nonscored,
    model_type="4l",
    str_condition=str_condition,
)

1.0 / 99.0 % clip : -3.97 / 3.918
1.0 / 99.0 % clip : -9.931 / 1.505
fold:5, epochs:80, batch_size:128, LR:0.001
Using model 4l with seed 123 for train_and_evaluate
Trained with 874 features. train.shape: (23814, 876). train_targets.shape: (23814, 207)


100%|███████████████████████████████████████████████████████████████████████████████████| 5/5 [02:41<00:00, 32.39s/it]


Our out of folds mean log loss score is 0.01639072415329617
Our out of folds log loss for our seed blend model is 0.01639072415329617


[2020-11-06 07:40:38] - model_type:4l, oof:0.0163907, train_flag:train
4l	cliping → scaling	0.0163907	train


In [26]:
str_condition = "cliping → fe_stats → scaling"
(
    train,
    train_targets,
    test,
    sample_submission,
    features,
    train_targets_nonscored,
) = load_data()
train, test = datasets.fe_clipping(train, test)
train, test, features = datasets.fe_stats(train, test, flag_add=False)
train, test, features = datasets.scaling(train, test)
_, _ = mlp_tf.run_mlp_tf_logger(
    train,
    test,
    train_targets,
    features,
    train_targets_nonscored,
    model_type="4l",
    str_condition=str_condition,
)

1.0 / 99.0 % clip : -3.97 / 3.918
1.0 / 99.0 % clip : -9.931 / 1.505
fold:5, epochs:80, batch_size:128, LR:0.001
Using model 4l with seed 123 for train_and_evaluate
Trained with 889 features. train.shape: (23814, 891). train_targets.shape: (23814, 207)


100%|███████████████████████████████████████████████████████████████████████████████████| 5/5 [02:33<00:00, 30.68s/it]


Our out of folds mean log loss score is 0.016394915265482254
Our out of folds log loss for our seed blend model is 0.016394915265482254


[2020-11-06 07:43:38] - model_type:4l, oof:0.0163949, train_flag:train
4l	cliping → fe_stats → scaling	0.0163949	train


In [27]:
str_condition = "no feature_eng + seeds=[5, 12]"
(
    train,
    train_targets,
    test,
    sample_submission,
    features,
    train_targets_nonscored,
) = load_data()
_, _ = mlp_tf.run_mlp_tf_logger(
    train,
    test,
    train_targets,
    features,
    train_targets_nonscored,
    seeds=[5, 12],
    model_type="4l",
    str_condition=str_condition,
)

fold:5, epochs:80, batch_size:128, LR:0.001
Using model 4l with seed 5 for train_and_evaluate
Trained with 874 features. train.shape: (23814, 876). train_targets.shape: (23814, 207)


100%|███████████████████████████████████████████████████████████████████████████████████| 5/5 [02:28<00:00, 29.72s/it]


Our out of folds mean log loss score is 0.016415164661213934
Using model 4l with seed 12 for train_and_evaluate
Trained with 874 features. train.shape: (23814, 876). train_targets.shape: (23814, 206)


100%|███████████████████████████████████████████████████████████████████████████████████| 5/5 [02:26<00:00, 29.31s/it]


Our out of folds mean log loss score is 0.016432959041340213
Our out of folds log loss for our seed blend model is 0.0163158942958022


[2020-11-06 07:48:52] - model_type:4l, oof:0.0163159, train_flag:train
4l	no feature_eng + seeds=[5, 12]	0.0163159	train


In [28]:
str_condition = "no feature_eng + p_min=0.001"
(
    train,
    train_targets,
    test,
    sample_submission,
    features,
    train_targets_nonscored,
) = load_data()
_, _ = mlp_tf.run_mlp_tf_logger(
    train,
    test,
    train_targets,
    features,
    train_targets_nonscored,
    model_type="4l",
    str_condition=str_condition,
    p_min=0.001,
)

fold:5, epochs:80, batch_size:128, LR:0.001
Using model 4l with seed 123 for train_and_evaluate
Trained with 874 features. train.shape: (23814, 876). train_targets.shape: (23814, 207)


100%|███████████████████████████████████████████████████████████████████████████████████| 5/5 [02:32<00:00, 30.49s/it]


Our out of folds mean log loss score is 0.016417860003448276
Our out of folds log loss for our seed blend model is 0.016417860003448276


[2020-11-06 07:51:40] - model_type:4l, oof:0.0164733, train_flag:train
4l	no feature_eng + p_min=0.001	0.0164733	train


In [29]:
str_condition = "no feature_eng + p_min=0.0001"
(
    train,
    train_targets,
    test,
    sample_submission,
    features,
    train_targets_nonscored,
) = load_data()
_, _ = mlp_tf.run_mlp_tf_logger(
    train,
    test,
    train_targets,
    features,
    train_targets_nonscored,
    model_type="4l",
    str_condition=str_condition,
    p_min=0.0001,
)

fold:5, epochs:80, batch_size:128, LR:0.001
Using model 4l with seed 123 for train_and_evaluate
Trained with 874 features. train.shape: (23814, 876). train_targets.shape: (23814, 207)


100%|███████████████████████████████████████████████████████████████████████████████████| 5/5 [02:48<00:00, 33.78s/it]


Our out of folds mean log loss score is 0.016418535034054025
Our out of folds log loss for our seed blend model is 0.016418535034054025


[2020-11-06 07:54:44] - model_type:4l, oof:0.0164171, train_flag:train
4l	no feature_eng + p_min=0.0001	0.0164171	train


In [30]:
str_condition = "no feature_eng + p_min=0.000012"
(
    train,
    train_targets,
    test,
    sample_submission,
    features,
    train_targets_nonscored,
) = load_data()
_, _ = mlp_tf.run_mlp_tf_logger(
    train,
    test,
    train_targets,
    features,
    train_targets_nonscored,
    model_type="4l",
    str_condition=str_condition,
    p_min=0.000012,
)

fold:5, epochs:80, batch_size:128, LR:0.001
Using model 4l with seed 123 for train_and_evaluate
Trained with 874 features. train.shape: (23814, 876). train_targets.shape: (23814, 207)


100%|███████████████████████████████████████████████████████████████████████████████████| 5/5 [02:38<00:00, 31.79s/it]


Our out of folds mean log loss score is 0.01642010286740012
Our out of folds log loss for our seed blend model is 0.01642010286740012


[2020-11-06 07:57:38] - model_type:4l, oof:0.0164193, train_flag:train
4l	no feature_eng + p_min=0.000012	0.0164193	train


In [31]:
str_condition = "feature_eng: RankGauss"
(
    train,
    train_targets,
    test,
    sample_submission,
    features,
    train_targets_nonscored,
) = load_data()
train, test = datasets.fe_quantile_transformer(train, test)
_, _ = mlp_tf.run_mlp_tf_logger(
    train,
    test,
    train_targets,
    features,
    train_targets_nonscored,
    model_type="4l",
    str_condition=str_condition,
)

fold:5, epochs:80, batch_size:128, LR:0.001
Using model 4l with seed 123 for train_and_evaluate
Trained with 874 features. train.shape: (23814, 876). train_targets.shape: (23814, 207)


100%|███████████████████████████████████████████████████████████████████████████████████| 5/5 [02:18<00:00, 27.67s/it]


Our out of folds mean log loss score is 0.01651488976653086
Our out of folds log loss for our seed blend model is 0.01651488976653086


[2020-11-06 08:00:19] - model_type:4l, oof:0.0165149, train_flag:train
4l	feature_eng: RankGauss	0.0165149	train


In [32]:
str_condition = "feature_eng: Kmean"
(
    train,
    train_targets,
    test,
    sample_submission,
    features,
    train_targets_nonscored,
) = load_data()
train, test, features = datasets.fe_cluster(train, test)
_, _ = mlp_tf.run_mlp_tf_logger(
    train,
    test,
    train_targets,
    features,
    train_targets_nonscored,
    model_type="4l",
    str_condition=str_condition,
)

fold:5, epochs:80, batch_size:128, LR:0.001
Using model 4l with seed 123 for train_and_evaluate
Trained with 914 features. train.shape: (23814, 916). train_targets.shape: (23814, 207)


100%|███████████████████████████████████████████████████████████████████████████████████| 5/5 [02:38<00:00, 31.68s/it]


Our out of folds mean log loss score is 0.016428480786353908
Our out of folds log loss for our seed blend model is 0.016428480786353908


[2020-11-06 08:03:35] - model_type:4l, oof:0.0164285, train_flag:train
4l	feature_eng: Kmean	0.0164285	train


In [33]:
str_condition = "feature_eng: c_abs"
(
    train,
    train_targets,
    test,
    sample_submission,
    features,
    train_targets_nonscored,
) = load_data()
train, test, features = datasets.c_abs(train, test)
_, _ = mlp_tf.run_mlp_tf_logger(
    train,
    test,
    train_targets,
    features,
    train_targets_nonscored,
    model_type="4l",
    str_condition=str_condition,
)

fold:5, epochs:80, batch_size:128, LR:0.001
Using model 4l with seed 123 for train_and_evaluate
Trained with 974 features. train.shape: (23814, 976). train_targets.shape: (23814, 207)


100%|███████████████████████████████████████████████████████████████████████████████████| 5/5 [02:42<00:00, 32.51s/it]


Our out of folds mean log loss score is 0.016341572635491387
Our out of folds log loss for our seed blend model is 0.016341572635491387


[2020-11-06 08:06:33] - model_type:4l, oof:0.0163416, train_flag:train
4l	feature_eng: c_abs	0.0163416	train


In [34]:
str_condition = "feature_eng: g_valid"
(
    train,
    train_targets,
    test,
    sample_submission,
    features,
    train_targets_nonscored,
) = load_data()
train, test, features = datasets.g_valid(train, test)
_, _ = mlp_tf.run_mlp_tf_logger(
    train,
    test,
    train_targets,
    features,
    train_targets_nonscored,
    model_type="4l",
    str_condition=str_condition,
)

fold:5, epochs:80, batch_size:128, LR:0.001
Using model 4l with seed 123 for train_and_evaluate
Trained with 1646 features. train.shape: (23814, 1648). train_targets.shape: (23814, 207)


100%|███████████████████████████████████████████████████████████████████████████████████| 5/5 [02:49<00:00, 33.94s/it]


Our out of folds mean log loss score is 0.01642308050404753
Our out of folds log loss for our seed blend model is 0.01642308050404753


[2020-11-06 08:10:14] - model_type:4l, oof:0.0164231, train_flag:train
4l	feature_eng: g_valid	0.0164231	train


In [35]:
str_condition = "feature_eng: g_abs"
(
    train,
    train_targets,
    test,
    sample_submission,
    features,
    train_targets_nonscored,
) = load_data()
train, test, features = datasets.g_abs(train, test)
_, _ = mlp_tf.run_mlp_tf_logger(
    train,
    test,
    train_targets,
    features,
    train_targets_nonscored,
    model_type="4l",
    str_condition=str_condition,
)

fold:5, epochs:80, batch_size:128, LR:0.001
Using model 4l with seed 123 for train_and_evaluate
Trained with 1646 features. train.shape: (23814, 1648). train_targets.shape: (23814, 207)


100%|███████████████████████████████████████████████████████████████████████████████████| 5/5 [02:34<00:00, 30.86s/it]


Our out of folds mean log loss score is 0.016526913096311505
Our out of folds log loss for our seed blend model is 0.016526913096311505


[2020-11-06 08:13:05] - model_type:4l, oof:0.0165269, train_flag:train
4l	feature_eng: g_abs	0.0165269	train


In [36]:
str_condition = "feature_eng: g_binary"
(
    train,
    train_targets,
    test,
    sample_submission,
    features,
    train_targets_nonscored,
) = load_data()
train, test, features = datasets.g_binary(train, test)
_, _ = mlp_tf.run_mlp_tf_logger(
    train,
    test,
    train_targets,
    features,
    train_targets_nonscored,
    model_type="4l",
    str_condition=str_condition,
)

fold:5, epochs:80, batch_size:128, LR:0.001
Using model 4l with seed 123 for train_and_evaluate
Trained with 1646 features. train.shape: (23814, 1648). train_targets.shape: (23814, 207)


100%|███████████████████████████████████████████████████████████████████████████████████| 5/5 [02:49<00:00, 33.81s/it]


Our out of folds mean log loss score is 0.016719939286481203
Our out of folds log loss for our seed blend model is 0.016719939286481203


[2020-11-06 08:16:15] - model_type:4l, oof:0.0167199, train_flag:train
4l	feature_eng: g_binary	0.0167199	train


In [37]:
str_condition = "feature_eng: c_binary"
(
    train,
    train_targets,
    test,
    sample_submission,
    features,
    train_targets_nonscored,
) = load_data()
train, test, features = datasets.c_binary(train, test)
_, _ = mlp_tf.run_mlp_tf_logger(
    train,
    test,
    train_targets,
    features,
    train_targets_nonscored,
    model_type="4l",
    str_condition=str_condition,
)

fold:5, epochs:80, batch_size:128, LR:0.001
Using model 4l with seed 123 for train_and_evaluate
Trained with 974 features. train.shape: (23814, 976). train_targets.shape: (23814, 207)


100%|███████████████████████████████████████████████████████████████████████████████████| 5/5 [02:17<00:00, 27.56s/it]


Our out of folds mean log loss score is 0.016473761854097265
Our out of folds log loss for our seed blend model is 0.016473761854097265


[2020-11-06 08:18:49] - model_type:4l, oof:0.0164738, train_flag:train
4l	feature_eng: c_binary	0.0164738	train


In [38]:
str_condition = "feature_eng: RankGauss → pca → variance_threshold"
(
    train,
    train_targets,
    test,
    sample_submission,
    features,
    train_targets_nonscored,
) = load_data()
train, test = datasets.fe_quantile_transformer(train, test)
train, test, features = datasets.fe_pca(
    train, test, n_components_g=600, n_components_c=50, SEED=42
)
train, test, features = datasets.fe_variance_threshold(train, test)
_, _ = mlp_tf.run_mlp_tf_logger(
    train,
    test,
    train_targets,
    features,
    train_targets_nonscored,
    model_type="4l",
    str_condition=str_condition,
)

fold:5, epochs:80, batch_size:128, LR:0.001
Using model 4l with seed 123 for train_and_evaluate
Trained with 1038 features. train.shape: (23814, 1040). train_targets.shape: (23814, 207)


100%|███████████████████████████████████████████████████████████████████████████████████| 5/5 [02:56<00:00, 35.25s/it]


Our out of folds mean log loss score is 0.01652331065609927
Our out of folds log loss for our seed blend model is 0.01652331065609927


[2020-11-06 08:22:13] - model_type:4l, oof:0.0165233, train_flag:train
4l	feature_eng: RankGauss → pca → variance_threshold	0.0165233	train


In [39]:
str_condition = "feature_eng: RankGauss → pca → variance_threshold → Kmean"
(
    train,
    train_targets,
    test,
    sample_submission,
    features,
    train_targets_nonscored,
) = load_data()
train, test = datasets.fe_quantile_transformer(train, test)
train, test, features = datasets.fe_pca(
    train, test, n_components_g=600, n_components_c=50, SEED=42
)
train, test, features = datasets.fe_variance_threshold(train, test)
train, test, features = datasets.fe_cluster(train, test)
_, _ = mlp_tf.run_mlp_tf_logger(
    train,
    test,
    train_targets,
    features,
    train_targets_nonscored,
    model_type="4l",
    str_condition=str_condition,
)

fold:5, epochs:80, batch_size:128, LR:0.001
Using model 4l with seed 123 for train_and_evaluate
Trained with 1078 features. train.shape: (23814, 1080). train_targets.shape: (23814, 207)


100%|███████████████████████████████████████████████████████████████████████████████████| 5/5 [02:43<00:00, 32.80s/it]


Our out of folds mean log loss score is 0.0164885426626139
Our out of folds log loss for our seed blend model is 0.0164885426626139


[2020-11-06 08:25:48] - model_type:4l, oof:0.0164885, train_flag:train
4l	feature_eng: RankGauss → pca → variance_threshold → Kmean	0.0164885	train


In [40]:
str_condition = "feature_eng: RankGauss → pca → variance_threshold → Kmean → fe_stats flag_add=False → c_squared"
(
    train,
    train_targets,
    test,
    sample_submission,
    features,
    train_targets_nonscored,
) = load_data()
train, test = datasets.fe_quantile_transformer(train, test)
train, test, features = datasets.fe_pca(
    train, test, n_components_g=600, n_components_c=50, SEED=42
)
train, test, features = datasets.fe_variance_threshold(train, test)
train, test, features = datasets.fe_cluster(train, test)
train, test, features = datasets.fe_stats(train, test, flag_add=False)
train, test, features = datasets.c_squared(train, test)
_, _ = mlp_tf.run_mlp_tf_logger(
    train,
    test,
    train_targets,
    features,
    train_targets_nonscored,
    model_type="4l",
    str_condition=str_condition,
)

fold:5, epochs:80, batch_size:128, LR:0.001
Using model 4l with seed 123 for train_and_evaluate
Trained with 1193 features. train.shape: (23814, 1195). train_targets.shape: (23814, 207)


100%|███████████████████████████████████████████████████████████████████████████████████| 5/5 [02:42<00:00, 32.47s/it]


Our out of folds mean log loss score is 0.016528910127349562
Our out of folds log loss for our seed blend model is 0.016528910127349562


[2020-11-06 08:29:28] - model_type:4l, oof:0.0165289, train_flag:train
4l	feature_eng: RankGauss → pca → variance_threshold → Kmean → fe_stats flag_add=False → c_squared	0.0165289	train


In [41]:
str_condition = "feature_eng: RankGauss → pca → variance_threshold → Kmean → scaling(RobustScaler)→ fe_stats flag_add=False → c_squared"
(
    train,
    train_targets,
    test,
    sample_submission,
    features,
    train_targets_nonscored,
) = load_data()
train, test = datasets.fe_quantile_transformer(train, test)
train, test, features = datasets.fe_pca(
    train, test, n_components_g=600, n_components_c=50, SEED=42
)
train, test, features = datasets.fe_variance_threshold(train, test)
train, test, features = datasets.fe_cluster(train, test)
train, test, features = datasets.scaling(train, test)
train, test, features = datasets.fe_stats(train, test, flag_add=False)
train, test, features = datasets.c_squared(train, test)
_, _ = mlp_tf.run_mlp_tf_logger(
    train,
    test,
    train_targets,
    features,
    train_targets_nonscored,
    model_type="4l",
    str_condition=str_condition,
)

fold:5, epochs:80, batch_size:128, LR:0.001
Using model 4l with seed 123 for train_and_evaluate
Trained with 1193 features. train.shape: (23814, 1195). train_targets.shape: (23814, 207)


100%|███████████████████████████████████████████████████████████████████████████████████| 5/5 [02:33<00:00, 30.69s/it]


Our out of folds mean log loss score is 0.016533685746245374
Our out of folds log loss for our seed blend model is 0.016533685746245374


[2020-11-06 08:33:07] - model_type:4l, oof:0.0165337, train_flag:train
4l	feature_eng: RankGauss → pca → variance_threshold → Kmean → scaling(RobustScaler)→ fe_stats flag_add=False → c_squared	0.0165337	train


In [42]:
str_condition = "feature_eng: RankGauss → pca → variance_threshold → Kmean → fe_stats flag_add=False → c_squared → c_abs → scaling(RobustScaler)"
(
    train,
    train_targets,
    test,
    sample_submission,
    features,
    train_targets_nonscored,
) = load_data()
train, test = datasets.fe_quantile_transformer(train, test)
train, test, features = datasets.fe_pca(
    train, test, n_components_g=600, n_components_c=50, SEED=42
)
train, test, features = datasets.fe_variance_threshold(train, test)
train, test, features = datasets.fe_cluster(train, test)
train, test, features = datasets.fe_stats(train, test, flag_add=False)
train, test, features = datasets.c_squared(train, test)
train, test, features = datasets.c_abs(train, test)
train, test, features = datasets.scaling(train, test)
_, _ = mlp_tf.run_mlp_tf_logger(
    train,
    test,
    train_targets,
    features,
    train_targets_nonscored,
    model_type="4l",
    str_condition=str_condition,
)

fold:5, epochs:80, batch_size:128, LR:0.001
Using model 4l with seed 123 for train_and_evaluate
Trained with 1293 features. train.shape: (23814, 1295). train_targets.shape: (23814, 207)


100%|███████████████████████████████████████████████████████████████████████████████████| 5/5 [02:19<00:00, 27.82s/it]


Our out of folds mean log loss score is 0.016493835431063784
Our out of folds log loss for our seed blend model is 0.016493835431063784


[2020-11-06 08:36:34] - model_type:4l, oof:0.0164938, train_flag:train
4l	feature_eng: RankGauss → pca → variance_threshold → Kmean → fe_stats flag_add=False → c_squared → c_abs → scaling(RobustScaler)	0.0164938	train


In [43]:
str_condition = "feature_eng: scaling(RobustScaler)"
(
    train,
    train_targets,
    test,
    sample_submission,
    features,
    train_targets_nonscored,
) = load_data()
train, test, features = datasets.scaling(train, test)
_, _ = mlp_tf.run_mlp_tf_logger(
    train,
    test,
    train_targets,
    features,
    train_targets_nonscored,
    model_type="4l",
    str_condition=str_condition,
)

fold:5, epochs:80, batch_size:128, LR:0.001
Using model 4l with seed 123 for train_and_evaluate
Trained with 874 features. train.shape: (23814, 876). train_targets.shape: (23814, 207)


100%|███████████████████████████████████████████████████████████████████████████████████| 5/5 [02:33<00:00, 30.74s/it]


Our out of folds mean log loss score is 0.016413690246593705
Our out of folds log loss for our seed blend model is 0.016413690246593705


[2020-11-06 08:39:26] - model_type:4l, oof:0.0164137, train_flag:train
4l	feature_eng: scaling(RobustScaler)	0.0164137	train


In [44]:
str_condition = "feature_eng: scaling(StandardScaler)"
(
    train,
    train_targets,
    test,
    sample_submission,
    features,
    train_targets_nonscored,
) = load_data()
train, test, features = datasets.scaling(train, test, scaler=StandardScaler())
_, _ = mlp_tf.run_mlp_tf_logger(
    train,
    test,
    train_targets,
    features,
    train_targets_nonscored,
    model_type="4l",
    str_condition=str_condition,
)

fold:5, epochs:80, batch_size:128, LR:0.001
Using model 4l with seed 123 for train_and_evaluate
Trained with 874 features. train.shape: (23814, 876). train_targets.shape: (23814, 207)


100%|███████████████████████████████████████████████████████████████████████████████████| 5/5 [02:19<00:00, 27.82s/it]


Our out of folds mean log loss score is 0.016415946929797816
Our out of folds log loss for our seed blend model is 0.016415946929797816


[2020-11-06 08:42:04] - model_type:4l, oof:0.0164159, train_flag:train
4l	feature_eng: scaling(StandardScaler)	0.0164159	train


In [45]:
str_condition = "feature_eng: fe_stats flag_add=True"
(
    train,
    train_targets,
    test,
    sample_submission,
    features,
    train_targets_nonscored,
) = load_data()
train, test, features = datasets.fe_stats(train, test, flag_add=True)
_, _ = mlp_tf.run_mlp_tf_logger(
    train,
    test,
    train_targets,
    features,
    train_targets_nonscored,
    model_type="4l",
    str_condition=str_condition,
)

fold:5, epochs:80, batch_size:128, LR:0.001
Using model 4l with seed 123 for train_and_evaluate
Trained with 898 features. train.shape: (23814, 900). train_targets.shape: (23814, 207)


100%|███████████████████████████████████████████████████████████████████████████████████| 5/5 [02:46<00:00, 33.31s/it]


Our out of folds mean log loss score is 0.01638532914324773
Our out of folds log loss for our seed blend model is 0.01638532914324773


[2020-11-06 08:45:17] - model_type:4l, oof:0.0163853, train_flag:train
4l	feature_eng: fe_stats flag_add=True	0.0163853	train


In [46]:
str_condition = "feature_eng: fe_stats flag_add=False"
(
    train,
    train_targets,
    test,
    sample_submission,
    features,
    train_targets_nonscored,
) = load_data()
train, test, features = datasets.fe_stats(train, test, flag_add=False)
_, _ = mlp_tf.run_mlp_tf_logger(
    train,
    test,
    train_targets,
    features,
    train_targets_nonscored,
    model_type="4l",
    str_condition=str_condition,
)

fold:5, epochs:80, batch_size:128, LR:0.001
Using model 4l with seed 123 for train_and_evaluate
Trained with 889 features. train.shape: (23814, 891). train_targets.shape: (23814, 207)


100%|███████████████████████████████████████████████████████████████████████████████████| 5/5 [02:31<00:00, 30.40s/it]


Our out of folds mean log loss score is 0.016398407289922035
Our out of folds log loss for our seed blend model is 0.016398407289922035


[2020-11-06 08:48:11] - model_type:4l, oof:0.0163984, train_flag:train
4l	feature_eng: fe_stats flag_add=False	0.0163984	train


In [47]:
str_condition = "feature_eng: fe_stats g only"
(
    train,
    train_targets,
    test,
    sample_submission,
    features,
    train_targets_nonscored,
) = load_data()
train, test, features = datasets.fe_stats(train, test, params=["g"])
_, _ = mlp_tf.run_mlp_tf_logger(
    train,
    test,
    train_targets,
    features,
    train_targets_nonscored,
    model_type="4l",
    str_condition=str_condition,
)

fold:5, epochs:80, batch_size:128, LR:0.001
Using model 4l with seed 123 for train_and_evaluate
Trained with 879 features. train.shape: (23814, 881). train_targets.shape: (23814, 207)


100%|███████████████████████████████████████████████████████████████████████████████████| 5/5 [02:25<00:00, 29.02s/it]


Our out of folds mean log loss score is 0.016411038796386595
Our out of folds log loss for our seed blend model is 0.016411038796386595


[2020-11-06 08:50:54] - model_type:4l, oof:0.016411, train_flag:train
4l	feature_eng: fe_stats g only	0.016411	train


In [48]:
str_condition = "feature_eng: fe_stats c only"
(
    train,
    train_targets,
    test,
    sample_submission,
    features,
    train_targets_nonscored,
) = load_data()
train, test, features = datasets.fe_stats(train, test, params=["c"])
_, _ = mlp_tf.run_mlp_tf_logger(
    train,
    test,
    train_targets,
    features,
    train_targets_nonscored,
    model_type="4l",
    str_condition=str_condition,
)

fold:5, epochs:80, batch_size:128, LR:0.001
Using model 4l with seed 123 for train_and_evaluate
Trained with 879 features. train.shape: (23814, 881). train_targets.shape: (23814, 207)


100%|███████████████████████████████████████████████████████████████████████████████████| 5/5 [02:36<00:00, 31.22s/it]


Our out of folds mean log loss score is 0.016420352734854742
Our out of folds log loss for our seed blend model is 0.016420352734854742


[2020-11-06 08:53:46] - model_type:4l, oof:0.0164204, train_flag:train
4l	feature_eng: fe_stats c only	0.0164204	train


In [49]:
str_condition = "feature_eng: fe_stats g,c"
(
    train,
    train_targets,
    test,
    sample_submission,
    features,
    train_targets_nonscored,
) = load_data()
train, test, features = datasets.fe_stats(train, test, params=["g", "c"])
_, _ = mlp_tf.run_mlp_tf_logger(
    train,
    test,
    train_targets,
    features,
    train_targets_nonscored,
    model_type="4l",
    str_condition=str_condition,
)

fold:5, epochs:80, batch_size:128, LR:0.001
Using model 4l with seed 123 for train_and_evaluate
Trained with 884 features. train.shape: (23814, 886). train_targets.shape: (23814, 207)


100%|███████████████████████████████████████████████████████████████████████████████████| 5/5 [02:51<00:00, 34.34s/it]


Our out of folds mean log loss score is 0.01637087886593117
Our out of folds log loss for our seed blend model is 0.01637087886593117


[2020-11-06 08:56:57] - model_type:4l, oof:0.0163709, train_flag:train
4l	feature_eng: fe_stats g,c	0.0163709	train


In [50]:
str_condition = "feature_eng: fe_stats gc only"
(
    train,
    train_targets,
    test,
    sample_submission,
    features,
    train_targets_nonscored,
) = load_data()
train, test, features = datasets.fe_stats(train, test, params=["gc"])
_, _ = mlp_tf.run_mlp_tf_logger(
    train,
    test,
    train_targets,
    features,
    train_targets_nonscored,
    model_type="4l",
    str_condition=str_condition,
)

fold:5, epochs:80, batch_size:128, LR:0.001
Using model 4l with seed 123 for train_and_evaluate
Trained with 879 features. train.shape: (23814, 881). train_targets.shape: (23814, 207)


100%|███████████████████████████████████████████████████████████████████████████████████| 5/5 [02:34<00:00, 30.94s/it]


Our out of folds mean log loss score is 0.016413121856755565
Our out of folds log loss for our seed blend model is 0.016413121856755565


[2020-11-06 08:59:49] - model_type:4l, oof:0.0164131, train_flag:train
4l	feature_eng: fe_stats gc only	0.0164131	train


In [51]:
str_condition = "feature_eng: scaling → fe_stats flag_add=True"
(
    train,
    train_targets,
    test,
    sample_submission,
    features,
    train_targets_nonscored,
) = load_data()
train, test, features = datasets.scaling(
    train, test
)  # fe_statsの後にするとinfエラーが出て実行できない(infないんだけどなあ。。。)
train, test, features = datasets.fe_stats(train, test, flag_add=True)
_, _ = mlp_tf.run_mlp_tf_logger(
    train,
    test,
    train_targets,
    features,
    train_targets_nonscored,
    model_type="4l",
    str_condition=str_condition,
)

fold:5, epochs:80, batch_size:128, LR:0.001
Using model 4l with seed 123 for train_and_evaluate
Trained with 898 features. train.shape: (23814, 900). train_targets.shape: (23814, 207)


100%|███████████████████████████████████████████████████████████████████████████████████| 5/5 [02:33<00:00, 30.73s/it]


Our out of folds mean log loss score is 0.016382322838011405
Our out of folds log loss for our seed blend model is 0.016382322838011405


[2020-11-06 09:02:53] - model_type:4l, oof:0.0163823, train_flag:train
4l	feature_eng: scaling → fe_stats flag_add=True	0.0163823	train


In [52]:
str_condition = "feature_eng: g_squared"
(
    train,
    train_targets,
    test,
    sample_submission,
    features,
    train_targets_nonscored,
) = load_data()
train, test, features = datasets.g_squared(train, test)
_, _ = mlp_tf.run_mlp_tf_logger(
    train,
    test,
    train_targets,
    features,
    train_targets_nonscored,
    model_type="4l",
    str_condition=str_condition,
)

fold:5, epochs:80, batch_size:128, LR:0.001
Using model 4l with seed 123 for train_and_evaluate
Trained with 1646 features. train.shape: (23814, 1648). train_targets.shape: (23814, 207)


 20%|████████████████▌                                                                  | 1/5 [01:11<04:46, 71.59s/it]


KeyboardInterrupt: 

In [ ]:
str_condition = "feature_eng: g_squared → scaling"
(
    train,
    train_targets,
    test,
    sample_submission,
    features,
    train_targets_nonscored,
) = load_data()
train, test, features = datasets.g_squared(train, test)
train, test, features = datasets.scaling(train, test)
_, _ = mlp_tf.run_mlp_tf_logger(
    train,
    test,
    train_targets,
    features,
    train_targets_nonscored,
    model_type="4l",
    str_condition=str_condition,
)

In [ ]:
str_condition = "feature_eng: c_squared"
(
    train,
    train_targets,
    test,
    sample_submission,
    features,
    train_targets_nonscored,
) = load_data()
train, test, features = datasets.c_squared(train, test)
_, _ = mlp_tf.run_mlp_tf_logger(
    train,
    test,
    train_targets,
    features,
    train_targets_nonscored,
    model_type="4l",
    str_condition=str_condition,
)

In [ ]:
str_condition = "feature_eng: c_squared → scaling"
(
    train,
    train_targets,
    test,
    sample_submission,
    features,
    train_targets_nonscored,
) = load_data()
train, test, features = datasets.c_squared(train, test)
train, test, features = datasets.scaling(train, test)
_, _ = mlp_tf.run_mlp_tf_logger(
    train,
    test,
    train_targets,
    features,
    train_targets_nonscored,
    model_type="4l",
    str_condition=str_condition,
)

In [ ]:
str_condition = "feature_eng: fe_pca"
(
    train,
    train_targets,
    test,
    sample_submission,
    features,
    train_targets_nonscored,
) = load_data()
train, test, features = datasets.fe_pca(
    train, test, n_components_g=70, n_components_c=10, SEED=123
)
_, _ = mlp_tf.run_mlp_tf_logger(
    train,
    test,
    train_targets,
    features,
    train_targets_nonscored,
    model_type="4l",
    str_condition=str_condition,
)

In [ ]:
str_condition = "feature_eng: fe_pca is_fit_train_only=True"
(
    train,
    train_targets,
    test,
    sample_submission,
    features,
    train_targets_nonscored,
) = load_data()
train, test, features = datasets.fe_pca(
    train, test, n_components_g=70, n_components_c=10, SEED=123, is_fit_train_only=True,
)
_, _ = mlp_tf.run_mlp_tf_logger(
    train,
    test,
    train_targets,
    features,
    train_targets_nonscored,
    model_type="4l",
    str_condition=str_condition,
)

In [ ]:
# pcaの値確認
(
    train,
    train_targets,
    test,
    sample_submission,
    features,
    train_targets_nonscored,
) = load_data()
train, test, features = datasets.fe_pca(
    train, test, n_components_g=70, n_components_c=10, SEED=123
)
train

In [ ]:
# pcaの値確認
(
    train,
    train_targets,
    test,
    sample_submission,
    features,
    train_targets_nonscored,
) = load_data()
train, test, features = datasets.fe_pca(
    train, test, n_components_g=70, n_components_c=10, SEED=123, is_fit_train_only=True,
)
train

In [ ]:
str_condition = "feature_eng: fe_pca → scaling(RobustScaler)"
(
    train,
    train_targets,
    test,
    sample_submission,
    features,
    train_targets_nonscored,
) = load_data()
train, test, features = datasets.fe_pca(
    train, test, n_components_g=70, n_components_c=10, SEED=123
)
train, test, features = datasets.scaling(train, test)
_, _ = mlp_tf.run_mlp_tf_logger(
    train,
    test,
    train_targets,
    features,
    train_targets_nonscored,
    model_type="4l",
    str_condition=str_condition,
)

In [ ]:
str_condition = "feature_eng: fe_pca is_fit_train_only=True → scaling(RobustScaler)"
(
    train,
    train_targets,
    test,
    sample_submission,
    features,
    train_targets_nonscored,
) = load_data()
train, test, features = datasets.fe_pca(
    train, test, n_components_g=70, n_components_c=10, SEED=123, is_fit_train_only=True,
)
train, test, features = datasets.scaling(train, test)
_, _ = mlp_tf.run_mlp_tf_logger(
    train,
    test,
    train_targets,
    features,
    train_targets_nonscored,
    model_type="4l",
    str_condition=str_condition,
)

In [ ]:
str_condition = "feature_eng: fe_pca → scaling(StandardScaler)"
(
    train,
    train_targets,
    test,
    sample_submission,
    features,
    train_targets_nonscored,
) = load_data()
train, test, features = datasets.fe_pca(
    train, test, n_components_g=70, n_components_c=10, SEED=123
)
train, test, features = datasets.scaling(train, test, scaler=StandardScaler())
_, _ = mlp_tf.run_mlp_tf_logger(
    train,
    test,
    train_targets,
    features,
    train_targets_nonscored,
    model_type="4l",
    str_condition=str_condition,
)

In [ ]:
def split_gc(train, test):
    """g-,c-で特徴量分ける"""
    features_g = list(train.columns[2:4]) + [
        col for col in train.columns if "g-" in col
    ]
    features_c = list(train.columns[2:4]) + [
        col for col in train.columns if "c-" in col
    ]
    start_predictors_g = [s for s in mlp_tf.start_predictors if "g-" in s]
    start_predictors_c = [s for s in mlp_tf.start_predictors if "c-" in s]
    return (
        train[features_g],
        test[features_g],
        train[features_c],
        test[features_c],
        features_g,
        features_c,
        start_predictors_g,
        start_predictors_c,
    )

In [ ]:
str_condition = "g- feature only"
(
    train,
    train_targets,
    test,
    sample_submission,
    features,
    train_targets_nonscored,
) = load_data()
train, test, _, _, features, _, start_predictors_g, _ = split_gc(train, test)
_, _ = mlp_tf.run_mlp_tf_logger(
    train,
    test,
    train_targets,
    features,
    train_targets_nonscored,
    model_type="4l",
    str_condition=str_condition,
)

In [ ]:
str_condition = "g- feature only → scaling"
(
    train,
    train_targets,
    test,
    sample_submission,
    features,
    train_targets_nonscored,
) = load_data()
train, test, _, _, features, _, start_predictors_g, _ = split_gc(train, test)
train, test, features = datasets.scaling(train, test)
_, _ = mlp_tf.run_mlp_tf_logger(
    train,
    test,
    train_targets,
    features,
    train_targets_nonscored,
    model_type="4l",
    str_condition=str_condition,
)

In [ ]:
str_condition = "c- feature only"
(
    train,
    train_targets,
    test,
    sample_submission,
    features,
    train_targets_nonscored,
) = load_data()
_, _, train, test, _, features, _, start_predictors_c = split_gc(train, test)
_, _ = mlp_tf.run_mlp_tf_logger(
    train,
    test,
    train_targets,
    features,
    train_targets_nonscored,
    model_type="4l",
    str_condition=str_condition,
)

In [ ]:
str_condition = "c- feature only → scaling"
(
    train,
    train_targets,
    test,
    sample_submission,
    features,
    train_targets_nonscored,
) = load_data()
_, _, train, test, _, features, _, start_predictors_c = split_gc(train, test)
train, test, features = datasets.scaling(train, test)
_, _ = mlp_tf.run_mlp_tf_logger(
    train,
    test,
    train_targets,
    features,
    train_targets_nonscored,
    model_type="4l",
    str_condition=str_condition,
)

In [ ]:
str_condition = "feature_eng: c_squared → scaling → fe_stats flag_add=True"
(
    train,
    train_targets,
    test,
    sample_submission,
    features,
    train_targets_nonscored,
) = load_data()
train, test, features = datasets.c_squared(train, test)
train, test, features = datasets.scaling(train, test)
train, test, features = datasets.fe_stats(train, test, flag_add=True)
_, _ = mlp_tf.run_mlp_tf_logger(
    train,
    test,
    train_targets,
    features,
    train_targets_nonscored,
    model_type="4l",
    str_condition=str_condition,
)

In [62]:
str_condition = "fe_stats flag_add=False → g_squared → c_squared → fe_pca → scaling"
(
    train,
    train_targets,
    test,
    sample_submission,
    features,
    train_targets_nonscored,
) = load_data()
train, test, features = datasets.fe_stats(train, test, flag_add=False)
train, test, features = datasets.g_squared(train, test)
train, test, features = datasets.c_squared(train, test)
train, test, features = datasets.fe_pca(
    train, test, n_components_g=70, n_components_c=10, SEED=123
)
train, test, features = datasets.scaling(train, test)
_, _ = mlp_tf.run_mlp_tf_logger(
    train,
    test,
    train_targets,
    features,
    train_targets_nonscored,
    model_type="4l",
    str_condition=str_condition,
)

fold:5, epochs:80, batch_size:128, LR:0.001
Using model 4l with seed 123 for train_and_evaluate
Trained with 1841 features. train.shape: (23814, 1843). train_targets.shape: (23814, 207)


100%|███████████████████████████████████████████████████████████████████████████████████| 5/5 [02:50<00:00, 34.13s/it]


Our out of folds mean log loss score is 0.016563169544878083
Our out of folds log loss for our seed blend model is 0.016563169544878083


[2020-11-06 09:30:41] - model_type:4l, oof:0.0165632, train_flag:train
4l	fe_stats flag_add=False → g_squared → c_squared → fe_pca → scaling	0.0165632	train


In [ ]:
str_condition = (
    "fe_stats flag_add=False → c_squared → c_abs → g_valid → fe_pca → scaling"
)
(
    train,
    train_targets,
    test,
    sample_submission,
    features,
    train_targets_nonscored,
) = load_data()
train, test, features = datasets.fe_stats(train, test, flag_add=False)
train, test, features = datasets.c_squared(train, test)
train, test, features = datasets.c_abs(train, test)
train, test, features = datasets.g_valid(train, test)
train, test, features = datasets.fe_pca(
    train, test, n_components_g=70, n_components_c=10, SEED=123
)
train, test, features = datasets.scaling(train, test)
_, _ = mlp_tf.run_mlp_tf_logger(
    train,
    test,
    train_targets,
    features,
    train_targets_nonscored,
    model_type="4l",
    str_condition=str_condition,
)

In [ ]:
str_condition = "fe_stats flag_add=False → c_squared → c_abs → g_valid → fe_pca is_fit_train_only=True → scaling"
(
    train,
    train_targets,
    test,
    sample_submission,
    features,
    train_targets_nonscored,
) = load_data()
train, test, features = datasets.fe_stats(train, test, flag_add=False)
train, test, features = datasets.c_squared(train, test)
train, test, features = datasets.c_abs(train, test)
train, test, features = datasets.g_valid(train, test)
train, test, features = datasets.fe_pca(
    train, test, n_components_g=70, n_components_c=10, SEED=123, is_fit_train_only=True,
)
train, test, features = datasets.scaling(train, test)
_, _ = mlp_tf.run_mlp_tf_logger(
    train,
    test,
    train_targets,
    features,
    train_targets_nonscored,
    model_type="4l",
    str_condition=str_condition,
)

In [ ]:
str_condition = "fe_stats flag_add=False → c_squared → fe_pca → scaling"
(
    train,
    train_targets,
    test,
    sample_submission,
    features,
    train_targets_nonscored,
) = load_data()
train, test, features = datasets.fe_stats(train, test, flag_add=False)
train, test, features = datasets.c_squared(train, test)
train, test, features = datasets.fe_pca(
    train, test, n_components_g=70, n_components_c=10, SEED=123
)
train, test, features = datasets.scaling(train, test)
_, _ = mlp_tf.run_mlp_tf_logger(
    train,
    test,
    train_targets,
    features,
    train_targets_nonscored,
    model_type="4l",
    str_condition=str_condition,
)
str_condition = "fe_stats flag_add=False → c_squared → c_abs → fe_pca → scaling"
(
    train,
    train_targets,
    test,
    sample_submission,
    features,
    train_targets_nonscored,
) = load_data()
train, test, features = datasets.fe_stats(train, test, flag_add=False)
train, test, features = datasets.c_squared(train, test)
train, test, features = datasets.c_abs(train, test)
train, test, features = datasets.fe_pca(
    train, test, n_components_g=70, n_components_c=10, SEED=123
)
train, test, features = datasets.scaling(train, test)
_, _ = mlp_tf.run_mlp_tf_logger(
    train,
    test,
    train_targets,
    features,
    train_targets_nonscored,
    model_type="4l",
    str_condition=str_condition,
)
str_condition = "fe_stats flag_add=False → c_squared → c_abs → fe_pca → RankGauss"
(
    train,
    train_targets,
    test,
    sample_submission,
    features,
    train_targets_nonscored,
) = load_data()
train, test, features = datasets.fe_stats(train, test, flag_add=False)
train, test, features = datasets.c_squared(train, test)
train, test, features = datasets.c_abs(train, test)
train, test, features = datasets.fe_pca(
    train, test, n_components_g=70, n_components_c=10, SEED=123
)
train, test = datasets.fe_quantile_transformer(train, test)
_, _ = mlp_tf.run_mlp_tf_logger(
    train,
    test,
    train_targets,
    features,
    train_targets_nonscored,
    model_type="4l",
    str_condition=str_condition,
)
str_condition = "fe_stats flag_add=True → c_squared → fe_pca → scaling"
(
    train,
    train_targets,
    test,
    sample_submission,
    features,
    train_targets_nonscored,
) = load_data()
train, test, features = datasets.fe_stats(train, test, flag_add=True)
train, test, features = datasets.c_squared(train, test)
train, test, features = datasets.fe_pca(
    train, test, n_components_g=70, n_components_c=10, SEED=123
)
train, test, features = datasets.scaling(train, test)
_, _ = mlp_tf.run_mlp_tf_logger(
    train,
    test,
    train_targets,
    features,
    train_targets_nonscored,
    model_type="4l",
    str_condition=str_condition,
)
str_condition = "fe_ctl_mean"
(
    train,
    train_targets,
    test,
    sample_submission,
    features,
    train_targets_nonscored,
) = load_data()
train, test, features = datasets.fe_ctl_mean(train, test)
_, _ = mlp_tf.run_mlp_tf_logger(
    train,
    test,
    train_targets,
    features,
    train_targets_nonscored,
    model_type="4l",
    str_condition=str_condition,
)
str_condition = "fe_ctl_mean gc_flg=g"
(
    train,
    train_targets,
    test,
    sample_submission,
    features,
    train_targets_nonscored,
) = load_data()
train, test, features = datasets.fe_ctl_mean(train, test, gc_flg="g")
_, _ = mlp_tf.run_mlp_tf_logger(
    train,
    test,
    train_targets,
    features,
    train_targets_nonscored,
    model_type="4l",
    str_condition=str_condition,
)
str_condition = "fe_ctl_mean gc_flg=c"
(
    train,
    train_targets,
    test,
    sample_submission,
    features,
    train_targets_nonscored,
) = load_data()
train, test, features = datasets.fe_ctl_mean(train, test, gc_flg="c")
_, _ = mlp_tf.run_mlp_tf_logger(
    train,
    test,
    train_targets,
    features,
    train_targets_nonscored,
    model_type="4l",
    str_condition=str_condition,
)
str_condition = "fe_ctl_mean is_mean=False"
(
    train,
    train_targets,
    test,
    sample_submission,
    features,
    train_targets_nonscored,
) = load_data()
train, test, features = datasets.fe_ctl_mean(train, test, is_mean=False)
_, _ = mlp_tf.run_mlp_tf_logger(
    train,
    test,
    train_targets,
    features,
    train_targets_nonscored,
    model_type="4l",
    str_condition=str_condition,
)

In [ ]:
str_condition = "fe_ctl_mean is_ratio=False"
(
    train,
    train_targets,
    test,
    sample_submission,
    features,
    train_targets_nonscored,
) = load_data()
train, test, features = datasets.fe_ctl_mean(train, test, is_ratio=False)
_, _ = mlp_tf.run_mlp_tf_logger(
    train,
    test,
    train_targets,
    features,
    train_targets_nonscored,
    model_type="4l",
    str_condition=str_condition,
)

In [ ]:
str_condition = "fe_ctl_mean → fe_variance_threshold"
(
    train,
    train_targets,
    test,
    sample_submission,
    features,
    train_targets_nonscored,
) = load_data()
train, test, features = datasets.fe_ctl_mean(train, test)
train, test, features = datasets.fe_variance_threshold(train, test)
_, _ = mlp_tf.run_mlp_tf_logger(
    train,
    test,
    train_targets,
    features,
    train_targets_nonscored,
    model_type="4l",
    str_condition=str_condition,
)

In [ ]:
str_condition = "fe_ctl_mean → scaling"
(
    train,
    train_targets,
    test,
    sample_submission,
    features,
    train_targets_nonscored,
) = load_data()
train, test, features = datasets.fe_ctl_mean(train, test)
train, test, features = datasets.scaling(train, test)
_, _ = mlp_tf.run_mlp_tf_logger(
    train,
    test,
    train_targets,
    features,
    train_targets_nonscored,
    model_type="4l",
    str_condition=str_condition,
)

In [ ]:
str_condition = "fe_ctl_mean → RankGauss"
(
    train,
    train_targets,
    test,
    sample_submission,
    features,
    train_targets_nonscored,
) = load_data()
train, test, features = datasets.fe_ctl_mean(train, test)
train, test = datasets.fe_quantile_transformer(train, test)
_, _ = mlp_tf.run_mlp_tf_logger(
    train,
    test,
    train_targets,
    features,
    train_targets_nonscored,
    model_type="4l",
    str_condition=str_condition,
)

In [ ]:
str_condition = "RankGauss → fe_ctl_mean → fe_variance_threshold"
(
    train,
    train_targets,
    test,
    sample_submission,
    features,
    train_targets_nonscored,
) = load_data()
train, test = datasets.fe_quantile_transformer(train, test)
train, test, features = datasets.fe_ctl_mean(train, test)
train, test, features = datasets.fe_variance_threshold(train, test)
_, _ = mlp_tf.run_mlp_tf_logger(
    train,
    test,
    train_targets,
    features,
    train_targets_nonscored,
    model_type="4l",
    str_condition=str_condition,
)

In [ ]:
str_condition = "fe_stats flag_add=False → c_squared → g_valid → fe_pca → scaling"
(
    train,
    train_targets,
    test,
    sample_submission,
    features,
    train_targets_nonscored,
) = load_data()
train, test, features = datasets.fe_stats(train, test, flag_add=False)
train, test, features = datasets.c_squared(train, test)
train, test, features = datasets.g_valid(train, test)
train, test, features = datasets.fe_pca(
    train, test, n_components_g=70, n_components_c=10, SEED=123
)
train, test, features = datasets.scaling(train, test)
_, _ = mlp_tf.run_mlp_tf_logger(
    train,
    test,
    train_targets,
    features,
    train_targets_nonscored,
    model_type="4l",
    str_condition=str_condition,
)

In [ ]:
str_condition = "fe_stats flag_add=False → c_squared → fe_ctl_mean → fe_pca → scaling"
(
    train,
    train_targets,
    test,
    sample_submission,
    features,
    train_targets_nonscored,
) = load_data()
train, test, features = datasets.fe_stats(train, test, flag_add=False)
train, test, features = datasets.c_squared(train, test)
train, test, features = datasets.fe_ctl_mean(train, test)
train, test, features = datasets.fe_pca(
    train, test, n_components_g=70, n_components_c=10, SEED=123
)
train, test, features = datasets.scaling(train, test)
_, _ = mlp_tf.run_mlp_tf_logger(
    train,
    test,
    train_targets,
    features,
    train_targets_nonscored,
    model_type="4l",
    str_condition=str_condition,
)
str_condition = "fe_stats flag_add=False → c_squared → g_valid → fe_pca → scaling"
(
    train,
    train_targets,
    test,
    sample_submission,
    features,
    train_targets_nonscored,
) = load_data()
train, test, features = datasets.fe_stats(train, test, flag_add=False)
train, test, features = datasets.c_squared(train, test)
train, test, features = datasets.g_valid(train, test)
train, test, features = datasets.fe_pca(
    train, test, n_components_g=70, n_components_c=10, SEED=123
)
train, test, features = datasets.scaling(train, test)
_, _ = mlp_tf.run_mlp_tf_logger(
    train,
    test,
    train_targets,
    features,
    train_targets_nonscored,
    model_type="4l",
    str_condition=str_condition,
)

In [ ]:
str_condition = (
    "targets_nonscored + fe_stats flag_add=False → c_squared  → fe_pca → scaling"
)
(
    train,
    train_targets,
    test,
    sample_submission,
    train_targets_nonscored,
    train_drug,
) = datasets.load_orig_data()
train, train_targets, test, train_targets_nonscored = datasets.mapping_and_filter(
    train, train_targets, test, train_targets_nonscored, is_conat_nonscore=True
)
train, test, features = datasets.fe_stats(train, test, flag_add=False)
train, test, features = datasets.c_squared(train, test)
train, test, features = datasets.fe_pca(
    train, test, n_components_g=70, n_components_c=10, SEED=123
)
train, test, features = datasets.scaling(train, test)
_, _ = mlp_tf.run_mlp_tf_logger(
    train,
    test,
    train_targets,
    features,
    train_targets_nonscored,
    model_type="4l",
    str_condition=str_condition,
)

In [61]:
str_condition = (
    "targets_nonscored + fe_stats flag_add=False → c_squared  → fe_pca → scaling"
)
(
    train,
    train_targets,
    test,
    sample_submission,
    train_targets_nonscored,
    train_drug,
) = datasets.load_orig_data()
train, train_targets, test, train_targets_nonscored = datasets.mapping_and_filter(
    train, train_targets, test, train_targets_nonscored, is_conat_nonscore=True
)
train, test, features = datasets.fe_stats(train, test, flag_add=False)
train, test, features = datasets.c_squared(train, test)
train, test, features = datasets.fe_pca(
    train, test, n_components_g=70, n_components_c=10, SEED=123
)
train, test, features = datasets.scaling(train, test)
_, _ = mlp_tf.run_mlp_tf_logger(
    train,
    test,
    train_targets,
    features,
    train_targets_nonscored,
    model_type="rs",
    str_condition=str_condition,
)

fold:5, epochs:80, batch_size:128, LR:0.001
Using model rs with seed 123 for train_and_evaluate
Trained with 1069 features. train.shape: (23814, 1071). train_targets.shape: (23814, 538)


100%|███████████████████████████████████████████████████████████████████████████████████| 5/5 [02:04<00:00, 24.80s/it]


Our out of folds mean log loss score is 0.01670923842194732
Our out of folds log loss for our seed blend model is 0.01670923842194732


[2020-11-06 09:27:16] - model_type:rs, oof:0.0167092, train_flag:train
rs	targets_nonscored + fe_stats flag_add=False → c_squared  → fe_pca → scaling	0.0167092	train


In [42]:
str_condition = "EPOCHS = 200 + start_predictors → fe_stats flag_add=False → c_squared → c_abs → g_valid → fe_pca → scaling"
(
    train,
    train_targets,
    test,
    sample_submission,
    train_targets_nonscored,
    train_drug,
) = datasets.load_orig_data()
train, train_targets, test, train_targets_nonscored = datasets.mapping_and_filter(
    train, train_targets, test, train_targets_nonscored, is_conat_nonscore=False
)

features_g, features_c = datasets.get_features_gc(
    train, top_feat_cols=mlp_tf.start_predictors
)

train, test, features = datasets.fe_stats(
    train, test, flag_add=False, features_g=features_g, features_c=features_c
)
train, test, features = datasets.c_squared(train, test, features_c=features_c)
train, test, features = datasets.c_abs(train, test, features_c=features_c)
train, test, features = datasets.g_valid(train, test, features_g=features_g)
train, test, features = datasets.fe_pca(
    train,
    test,
    n_components_g=70,
    n_components_c=10,
    SEED=123,
    features_g=features_g,
    features_c=features_c,
)
train, test, features = datasets.scaling(train, test)

mlp_tf.EPOCHS = 200
_, _ = mlp_tf.run_mlp_tf_logger(
    train,
    test,
    train_targets,
    features,
    train_targets_nonscored,
    model_type="stack_tabnet",
    str_condition=str_condition,
)

fold:5, epochs:200, batch_size:128, LR:0.001
Using model stack_tabnet with seed 123 for train_and_evaluate
Trained with 1516 features. train.shape: (23814, 1518). train_targets.shape: (23814, 207)


  0%|                                                                                           | 0/5 [00:00<?, ?it/s]

params: None
Please check your arguments if you have upgraded adabelief-tf from version 0.0.1.
Modifications to default arguments:
                           eps  weight_decouple    rectify
-----------------------  -----  -----------------  -------------
adabelief-tf=0.0.1       1e-08  Not supported      Not supported
Current version (0.1.0)  1e-14  supported          default: True
For a complete table of recommended hyperparameters, see
https://github.com/juntang-zhuang/Adabelief-Optimizer



 20%|████████████████▍                                                                 | 1/5 [03:03<12:14, 183.52s/it]

params: None
Please check your arguments if you have upgraded adabelief-tf from version 0.0.1.
Modifications to default arguments:
                           eps  weight_decouple    rectify
-----------------------  -----  -----------------  -------------
adabelief-tf=0.0.1       1e-08  Not supported      Not supported
Current version (0.1.0)  1e-14  supported          default: True
For a complete table of recommended hyperparameters, see
https://github.com/juntang-zhuang/Adabelief-Optimizer



...

 80%|█████████████████████████████████████████████████████████████████▌                | 4/5 [11:57<03:00, 180.76s/it]

params: None
Please check your arguments if you have upgraded adabelief-tf from version 0.0.1.
Modifications to default arguments:
                           eps  weight_decouple    rectify
-----------------------  -----  -----------------  -------------
adabelief-tf=0.0.1       1e-08  Not supported      Not supported
Current version (0.1.0)  1e-14  supported          default: True
For a complete table of recommended hyperparameters, see
https://github.com/juntang-zhuang/Adabelief-Optimizer



100%|██████████████████████████████████████████████████████████████████████████████████| 5/5 [15:02<00:00, 180.52s/it]


Our out of folds mean log loss score is 0.016937076643232404
Our out of folds log loss for our seed blend model is 0.016937076643232404


[2020-11-06 12:22:25] - model_type:stack_tabnet, oof:0.0169371, train_flag:train
stack_tabnet	EPOCHS = 200 + start_predictors → fe_stats flag_add=False → c_squared → c_abs → g_valid → fe_pca → scaling	0.0169371	train


In [43]:
str_condition = "start_predictors → fe_stats flag_add=False → c_squared → c_abs → g_valid → fe_pca → scaling"
(
    train,
    train_targets,
    test,
    sample_submission,
    train_targets_nonscored,
    train_drug,
) = datasets.load_orig_data()
train, train_targets, test, train_targets_nonscored = datasets.mapping_and_filter(
    train, train_targets, test, train_targets_nonscored, is_conat_nonscore=False
)

features_g, features_c = datasets.get_features_gc(
    train, top_feat_cols=mlp_tf.start_predictors
)

train, test, features = datasets.fe_stats(
    train, test, flag_add=False, features_g=features_g, features_c=features_c
)
train, test, features = datasets.c_squared(train, test, features_c=features_c)
train, test, features = datasets.c_abs(train, test, features_c=features_c)
train, test, features = datasets.g_valid(train, test, features_g=features_g)
train, test, features = datasets.fe_pca(
    train,
    test,
    n_components_g=70,
    n_components_c=10,
    SEED=123,
    features_g=features_g,
    features_c=features_c,
)
train, test, features = datasets.scaling(train, test)

mlp_tf.EPOCHS = 80
_, _ = mlp_tf.run_mlp_tf_logger(
    train,
    test,
    train_targets,
    features,
    train_targets_nonscored,
    model_type="4l",
    str_condition=str_condition,
)

fold:5, epochs:80, batch_size:128, LR:0.001
Using model 4l with seed 123 for train_and_evaluate
Trained with 1516 features. train.shape: (23814, 1518). train_targets.shape: (23814, 207)


100%|███████████████████████████████████████████████████████████████████████████████████| 5/5 [02:46<00:00, 33.21s/it]


Our out of folds mean log loss score is 0.016312127472810176
Our out of folds log loss for our seed blend model is 0.016312127472810176


[2020-11-06 12:25:54] - model_type:4l, oof:0.0163121, train_flag:train
4l	start_predictors → fe_stats flag_add=False → c_squared → c_abs → g_valid → fe_pca → scaling	0.0163121	train


In [ ]:
mlp_tf.LR = 0.001
mlp_tf.BATCH_SIZE = 256
mlp_tf.EPOCHS = 80

str_condition = "no feature_eng + batch_size=256"
(
    train,
    train_targets,
    test,
    sample_submission,
    features,
    train_targets_nonscored,
) = load_data()
_, _ = mlp_tf.run_mlp_tf_logger(
    train,
    test,
    train_targets,
    features,
    train_targets_nonscored,
    model_type="4l",
    str_condition=str_condition,
)

In [ ]:
mlp_tf.LR = 0.001
mlp_tf.BATCH_SIZE = 1024
mlp_tf.EPOCHS = 80

str_condition = "no feature_eng + batch_size=1024"
(
    train,
    train_targets,
    test,
    sample_submission,
    features,
    train_targets_nonscored,
) = load_data()
_, _ = mlp_tf.run_mlp_tf_logger(
    train,
    test,
    train_targets,
    features,
    train_targets_nonscored,
    model_type="4l",
    str_condition=str_condition,
)

In [ ]:
mlp_tf.LR = 0.001
mlp_tf.BATCH_SIZE = 64
mlp_tf.EPOCHS = 80

str_condition = "no feature_eng + batch_size=64"
(
    train,
    train_targets,
    test,
    sample_submission,
    features,
    train_targets_nonscored,
) = load_data()
_, _ = mlp_tf.run_mlp_tf_logger(b
    train,
    test,
    train_targets,
    features,
    train_targets_nonscored,
    model_type="4l",
    str_condition=str_condition,
)

In [ ]:
mlp_tf.LR = 0.001
mlp_tf.BATCH_SIZE = 32
mlp_tf.EPOCHS = 80

str_condition = "no feature_eng + batch_size=32"
(
    train,
    train_targets,
    test,
    sample_submission,
    features,
    train_targets_nonscored,
) = load_data()
_, _ = mlp_tf.run_mlp_tf_logger(
    train,
    test,
    train_targets,
    features,
    train_targets_nonscored,
    model_type="4l",
    str_condition=str_condition,
)

In [ ]:
mlp_tf.LR = 0.01
mlp_tf.BATCH_SIZE = 128
mlp_tf.EPOCHS = 80

str_condition = "no feature_eng + LR=0.01"
(
    train,
    train_targets,
    test,
    sample_submission,
    features,
    train_targets_nonscored,
) = load_data()
_, _ = mlp_tf.run_mlp_tf_logger(
    train,
    test,
    train_targets,
    features,
    train_targets_nonscored,
    model_type="4l",
    str_condition=str_condition,
)

In [ ]:
mlp_tf.LR = 0.01
mlp_tf.BATCH_SIZE = 128
mlp_tf.EPOCHS = 100

str_condition = "no feature_eng + LR=0.01 + epochs=100"
(
    train,
    train_targets,
    test,
    sample_submission,
    features,
    train_targets_nonscored,
) = load_data()
_, _ = mlp_tf.run_mlp_tf_logger(
    train,
    test,
    train_targets,
    features,
    train_targets_nonscored,
    model_type="4l",
    str_condition=str_condition,
)

In [ ]:
mlp_tf.LR = 0.1
mlp_tf.BATCH_SIZE = 128
mlp_tf.EPOCHS = 80

str_condition = "no feature_eng + LR=0.1"
(
    train,
    train_targets,
    test,
    sample_submission,
    features,
    train_targets_nonscored,
) = load_data()
_, _ = mlp_tf.run_mlp_tf_logger(
    train,
    test,
    train_targets,
    features,
    train_targets_nonscored,
    model_type="4l",
    str_condition=str_condition,
)

In [ ]:
mlp_tf.LR = 0.001
mlp_tf.BATCH_SIZE = 128
mlp_tf.EPOCHS = 80

In [37]:
def nelder_mead_weights_class(y_true: np.ndarray, oof_preds: list, method="L-BFGS-B"):
    """ネルダーミードでモデルのクラスごとのブレンド重み最適化
    ネルダーミード遅いから、L-BFGS-B をデフォルトの方法にする.重みクラス数文あるのでめちゃめちゃ時間かかる。。。"""
    from scipy.optimize import minimize

    def opt(ws, y_true, y_preds):
        y_pred = None
        for y_p in y_preds:
            if y_pred is None:
                y_pred = ws * y_p
            else:
                y_pred += ws * y_p
        return mlp_tf.mean_log_loss(y_true, y_pred)

    initial_weights = np.array([1.0 / y_true.shape[1]] * y_true.shape[1])
    if method in ["L-BFGS-B", "TNC", "COBYLA ", "SLSQP"]:
        # パラメータの範囲に制約のある方法で最適化
        # 1位の人は L-BFGS-B 使ってたので
        # https://www.kaggle.com/gogo827jz/optimise-blending-weights-with-bonus-0
        # パラメータの範囲は bounds に，最小値と最大値の対の系列を指定する必要あり
        # http://www.kamishima.net/mlmpyja/lr/optimization.html
        bnds = [(0, 1) for _ in range(y_true.shape[1])]
        result = minimize(
            opt,
            x0=initial_weights,
            args=(y_true, oof_preds),
            method=method,
            bounds=bnds,
        )
    else:
        result = minimize(
            opt, x0=initial_weights, args=(y_true, oof_preds), method=method
        )
    best_weights = result.x
    return best_weights


if "sig_id" in train_targets.columns:
    train_targets = train_targets.drop(["sig_id"], axis=1)
best_weights = nelder_mead_weights_class(train_targets.values,  [oof_pred, oof_pred * 2, oof_pred * 3])
best_weights

KeyboardInterrupt: 

## run_mlp_tf_blend_logger

In [10]:
str_condition = "posi_ratio_class_weight"

(
    train,
    train_targets,
    test,
    sample_submission,
    train_targets_nonscored,
    train_drug,
) = datasets.load_orig_data()
train, train_targets, test, train_targets_nonscored = datasets.mapping_and_filter(
    train, train_targets, test, train_targets_nonscored
)
features = datasets.get_features(train)

mlp_tf.run_mlp_tf_blend_logger(
    train,
    test,
    train_targets,
    features,
    train_targets_nonscored,
    model_types=[
        "2l",
        "3l",
        "4l",
        "5l",
        "rs",
        "stack_tabnet",
        "tabnet_class",
    ],
    str_condition=str_condition,
    class_weight = datasets.fetch_posi_ratio_class_weight(train_targets),
)

fold:5, epochs:80, batch_size:128, LR:0.001
Using model 2l with seed 123 for train_and_evaluate
Trained with 874 features. train.shape: (23814, 876). train_targets.shape: (23814, 207)


100%|███████████████████████████████████████████████████████████████████████████████████| 5/5 [03:10<00:00, 38.10s/it]


Our out of folds mean log loss score is 0.020377357356507884
Our out of folds log loss for our seed blend model is 0.020377357356507884


[2020-11-06 09:47:58] - model_type:2l, oof:0.0203774, train_flag:train
2l	posi_ratio_class_weight	0.0203774	train


fold:5, epochs:80, batch_size:128, LR:0.001
Using model 3l with seed 123 for train_and_evaluate
Trained with 874 features. train.shape: (23814, 876). train_targets.shape: (23814, 207)


...

 80%|██████████████████████████████████████████████████████████████████▍                | 4/5 [01:55<00:29, 29.27s/it]

params: None
Please check your arguments if you have upgraded adabelief-tf from version 0.0.1.
Modifications to default arguments:
                           eps  weight_decouple    rectify
-----------------------  -----  -----------------  -------------
adabelief-tf=0.0.1       1e-08  Not supported      Not supported
Current version (0.1.0)  1e-14  supported          default: True
For a complete table of recommended hyperparameters, see
https://github.com/juntang-zhuang/Adabelief-Optimizer



100%|███████████████████████████████████████████████████████████████████████████████████| 5/5 [02:28<00:00, 29.71s/it]


Our out of folds mean log loss score is 0.019247927762780635
Our out of folds log loss for our seed blend model is 0.019247927762780635


[2020-11-06 10:08:48] - model_type:tabnet_class, oof:0.0192479, train_flag:train
tabnet_class	posi_ratio_class_weight	0.0192479	train
[2020-11-06 10:08:51] - model_type:2l-3l-4l-5l-rs-stack_tabnet-tabnet_class, oof:0.0222129, train_flag:train
2l-3l-4l-5l-rs-stack_tabnet-tabnet_class	posi_ratio_class_weight:Mean blend	0.0222129	train


In [11]:
str_condition = "no feature_eng + posi_ratio_class_weight + targets_nonscored"

(
    train,
    train_targets,
    test,
    sample_submission,
    train_targets_nonscored,
    train_drug,
) = datasets.load_orig_data()
train, train_targets, test, train_targets_nonscored = datasets.mapping_and_filter(
    train, train_targets, test, train_targets_nonscored, is_conat_nonscore=True
)
features = datasets.get_features(train)

mlp_tf.run_mlp_tf_blend_logger(
    train,
    test,
    train_targets,
    features,
    train_targets_nonscored,
    model_types=[
        "2l",
        "3l",
        "4l",
        "5l",
        "rs",
        "stack_tabnet",
        "tabnet_class",
    ],
    str_condition=str_condition,
    class_weight = datasets.fetch_posi_ratio_class_weight(train_targets),
)

fold:5, epochs:80, batch_size:128, LR:0.001
Using model 2l with seed 123 for train_and_evaluate
Trained with 874 features. train.shape: (23814, 876). train_targets.shape: (23814, 538)


 40%|█████████████████████████████████▏                                                 | 2/5 [02:21<03:32, 70.90s/it]


KeyboardInterrupt: 

In [44]:
for lr in [0.1, 0.01, 0.03, 0.0001, 0.00001]:
    mlp_tf.LR = lr
    str_condition = f"no feature_eng + learning rate={mlp_tf.LR}"
    (
        train,
        train_targets,
        test,
        sample_submission,
        features,
        train_targets_nonscored,
    ) = load_data()
    mlp_tf.run_mlp_tf_blend_logger(
        train,
        test,
        train_targets,
        features,
        train_targets_nonscored,
        str_condition=str_condition,
        model_types=[
            "2l",
            "3l",
            "4l",
            "5l",
            "rs",
            "3l_v2",
        ],
    )
    
mlp_tf.LR = 0.001

fold:5, epochs:80, batch_size:128, LR:0.1
Using model 2l with seed 123 for train_and_evaluate
Trained with 874 features. train.shape: (23814, 876). train_targets.shape: (23814, 207)


100%|███████████████████████████████████████████████████████████████████████████████████| 5/5 [02:17<00:00, 27.53s/it]


Our out of folds mean log loss score is 0.01629360464586551
Our out of folds log loss for our seed blend model is 0.01629360464586551


[2020-11-06 12:28:26] - model_type:2l, oof:0.0162936, train_flag:train
2l	no feature_eng + learning rate=0.1	0.0162936	train


fold:5, epochs:80, batch_size:128, LR:0.1
Using model 3l with seed 123 for train_and_evaluate
Trained with 874 features. train.shape: (23814, 876). train_targets.shape: (23814, 207)


...

 80%|█████████████████████████████████████████████████████████████████▌                | 4/5 [10:22<02:35, 155.81s/it]

Please check your arguments if you have upgraded adabelief-tf from version 0.0.1.
Modifications to default arguments:
                           eps  weight_decouple    rectify
-----------------------  -----  -----------------  -------------
adabelief-tf=0.0.1       1e-08  Not supported      Not supported
Current version (0.1.0)  1e-14  supported          default: True
For a complete table of recommended hyperparameters, see
https://github.com/juntang-zhuang/Adabelief-Optimizer



100%|██████████████████████████████████████████████████████████████████████████████████| 5/5 [12:59<00:00, 155.92s/it]


Our out of folds mean log loss score is 0.017427688232432625
Our out of folds log loss for our seed blend model is 0.017427688232432625


[2020-11-06 14:55:59] - model_type:3l_v2, oof:0.0174277, train_flag:train
3l_v2	no feature_eng + learning rate=1e-05	0.0174277	train
[2020-11-06 14:56:02] - model_type:2l-3l-4l-5l-rs-3l_v2, oof:0.0381998, train_flag:train
2l-3l-4l-5l-rs-3l_v2	no feature_eng + learning rate=1e-05:Mean blend	0.0381998	train


In [45]:
for bs in [8, 32, 128, 512, 1024]:
    mlp_tf.BATCH_SIZE = bs
    str_condition = f"no feature_eng + mlp_tf.BATCH_SIZE={mlp_tf.BATCH_SIZE}"
    (
        train,
        train_targets,
        test,
        sample_submission,
        features,
        train_targets_nonscored,
    ) = load_data()
    mlp_tf.run_mlp_tf_blend_logger(
        train,
        test,
        train_targets,
        features,
        train_targets_nonscored,
        str_condition=str_condition,
        model_types=[
            "2l",
            "3l",
            "4l",
            "5l",
            "rs",
            "3l_v2",
        ],
    )
    
mlp_tf.BATCH_SIZE = 128

fold:5, epochs:80, batch_size:8, LR:0.001
Using model 2l with seed 123 for train_and_evaluate
Trained with 874 features. train.shape: (23814, 876). train_targets.shape: (23814, 207)


 40%|████████████████████████████████▊                                                 | 2/5 [20:37<30:55, 618.65s/it]


KeyboardInterrupt: 

In [ ]:
from IPython.display import HTML

HTML(
    """
<button id="code-show-switch-btn">スクリプトを非表示にする</button>

<script>
var code_show = true;

function switch_display_setting() {
    var switch_btn = $("#code-show-switch-btn");
    if (code_show) {
        $("div.input").hide();
        code_show = false;
        switch_btn.text("スクリプトを表示する");
    }else {
        $("div.input").show();
        code_show = true;
        switch_btn.text("スクリプトを非表示にする");
    }
}

$("#code-show-switch-btn").click(switch_display_setting);
</script>
"""
)